In [1]:
# 自动计算cell的计算时间
%load_ext autotime

time: 466 µs (started: 2021-08-05 11:14:50 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch11 #2 change Aug 05, 2021'

git push origin master

In [3]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

time: 6.28 s (started: 2021-08-05 11:15:01 +08:00)


In [4]:
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 22.2 ms (started: 2021-08-05 11:15:07 +08:00)


# 文本深度学习

> **本章包含**
> * 为机器学习应用程序预处理文本数据
> * 用于文本处理的词袋方法和序列建模方法
> * 变压器架构
> * 序列到序列学习

## 自然语言处理：鸟瞰图

在计算机科学中，我们将人类语言（如英语或普通话）称为“自然”语言，以区别于专为机器设计的语言，如汇编、LISP 或 XML。每一种机器语言都是：它的起点是人类工程师设计的写下一组正式规则来描述你可以用这种语言做出什么陈述，以及它们的含义。规则首先出现，人们只有在规则集完成后才开始使用该语言。对于人类语言，情况正好相反：使用在先，规则在后。自然语言是由进化过程塑造的，就像生物有机体一样——这就是它“自然”的原因。它的“规则”，就像英语的语法一样，事后正式化，经常被用户忽视或破坏。因此，虽然机器可读语言是高度结构化和严谨的，使用精确的句法规则从固定的词汇表中将精确定义的概念编织在一起，但自然语言是混乱的——模棱两可、混乱、庞大且不断变化。

创建能够理解自然语言的算法是一件大事：语言，尤其是文本，是我们大部分交流和大部分文化生产的基础。互联网主要是文本。语言是我们存储几乎所有知识的方式。我们的思想很大程度上建立在语言上。然而，理解自然语言的能力长期以来一直是机器所不具备的。有些人曾经天真地认为你可以简单地写下“英语规则集”，就像可以写下LISP的规则集一样。因此，构建自然语言处理 (NLP) 系统的早期尝试是通过“应用语言学”的视角进行的。工程师和语言学家会手工制作复杂的规则集来执行基本的机器翻译或创建简单的聊天机器人——比如 1960 年代著名的 ELIZA 程序，它使用模式匹配来维持非常基本的对话。但是语言是一种叛逆的东西，它不容易被形式化。经过几十年的努力，这些系统的能力仍然令人失望。

手工制定的规则一直是 1990 年代的主导方法。但从 1980 年代后期开始，更快的计算机和更高的数据可用性开始使更好的替代方案变得可行。当您发现自己构建的系统包含大量临时规则时，作为一名聪明的工程师，您可能会开始问：我可以使用数据语料库来自动化查找这些规则的过程吗？我可以在某种规则空间中搜索规则，而不必自己想出来吗？就这样，你已经毕业从事机器学习。所以在 1980 年代后期，我们开始看到机器学习方法用于自然语言处理。最早的那些是基于决策树的——其意图实际上是使以前系统的那种 if/then/else 规则的开发自动化。然后统计方法开始加速，从逻辑回归开始。随着时间的推移，学习的参数模型完全接管了，语言学开始被视为更多的障碍而不是有用的工具。早期的语音识别研究员 Frederick Jelinek 在 1990 年代开玩笑说：“每次我解雇一名语言学家，语音识别器的性能都会上升”。

这就是现代 NLP 的意义所在：使用机器学习和大型数据集赋予计算机能力——不是语言，这是一个更崇高的目标——而是摄取一段理解语言作为输入并返回一些有用的东西，比如预测：
* “这篇课文的主题是什么？” （文字分类）
* “这段文字是否包含辱骂？” （内容过滤）
* “这段文字听起来是正面的还是负面的？” （情绪分析）
* “这不完整的句子中的下一个单词应该是什么？” （语言建模）
* “这个用德语怎么说？” （翻译）
* “你如何用一段话概括这篇文章？” （总结）
* 等等。

当然，在本章中请记住，您将训练的文本处理模型并不具备人类对语言的理解； 相反，他们只是在输入数据中寻找统计规律——结果证明这足以在许多简单的任务上表现良好。 就像计算机视觉是应用于像素的模式识别一样，NLP 是应用于单词、句子和段落的模式识别。

NLP 的工具集——决策树、逻辑回归——从 1990 年代到 2010 年代初期才经历了缓慢的演变。 大多数研究重点是特征工程。 当我在 2013 年在 Kaggle 上赢得我的第一场 NLP 比赛时，我的模型，你猜对了，基于决策树和逻辑回归。 然而，在 2014-2015 年左右，事情终于开始发生变化。 多位研究人员开始研究循环神经网络的语言理解能力，特别是 LSTM——一种 1990 年代后期的序列处理算法，直到那时才受到关注

2015 年初，Keras 推出了第一个开源、易于使用的 LSTM 实现，这恰好是对循环神经网络重新产生兴趣的浪潮的开始——直到那时，只有“研究代码” 不能轻易重复使用。 然后从 2015 年到 2017 年，循环神经网络主导了蓬勃发展的 NLP 场景。 特别是双向 LSTM 模型在许多重要任务上设定了最新技术，从摘要到问答再到机器翻译。

最后，在 2017-2018 年左右，出现了一种取代 RNN 的新架构：Transformer，您将在本章的后半部分了解它。 Transformers 在短时间内在整个领域取得了长足的进步，如今大多数 NLP 系统都是基于它们的。

让我们深入了解细节。 本章将带您从最基础的知识到使用 Transformer 进行机器翻译。

## 准备文本数据

作为可微函数的深度学习模型只能处理数字张量：它们不能将原始文本作为输入。 text 是将文本转换为数字向量化张量的过程。 文本矢量化过程有多种形状和形式，但它们都遵循相同的模板：
* 首先，您将文本标准化以使其更易于处理，例如将其转换为小写或删除标点符号。
* 您将文本拆分为多个单元（称为“标记”），例如字符、单词或单词组。 这称为标记化。
* 您将每个这样的标记转换为数字向量。 这通常涉及首先索引数据中存在的所有标记。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt1np624r0j310a0s2jte.jpg)

让我们回顾一下每个步骤。

### 文本标准化

考虑这两个句子：
* “日落来了。我盯着墨西哥的天空。大自然是不是很壮观？？”
* “日落来了，我凝视着墨西哥的天空。大自然是不是很精彩？”

它们非常相似——事实上，它们几乎完全相同。 然而，如果您将它们转换为字节字符串，它们最终会得到非常不同的表示，因为“i”和“I”是两个不同的字符，“Mexico”和“México”是两个不同的词，“isnt”不是 't “不是”，等等。 机器学习模型不知道“i”和“I”是同一个字母，“é”是带重音的“e”，或者“凝视”和“凝视”是两种形式的 同一个动词。

文本标准化是特征工程的一种基本形式，旨在消除您不希望模型必须处理的编码差异。 它也不是机器学习独有的——如果你正在构建一个搜索引擎，你必须做同样的事情。

最简单和最广泛的标准化方案之一是“转换为小写并删除标点符号”。 我们的两句话会变成：
* “日落来了，我凝视着墨西哥的天空，这不是大自然的壮丽景色”
* “日落来了，我凝视着墨西哥的天空，大自然并不灿烂”

已经很近了。 另一种常见的转换是将特殊字符转换为标准形式，例如将“é”替换为“e”，将“æ”替换为“ae”等。 我们的代币“méxico”将变成“mexico”。

最后，在机器学习上下文中很少使用的更高级的标准化模式是词干：将术语的变体（例如动词的不同共轭词干形式）转换为单个共享表示，例如将“捕获”转换为“捕获” 和“一直在捕捉”变成“[catch]”或“cats”变成“[cat]”。 通过词干提取，“was staring”和“stared”会变成类似于“[stare]”的东西——我们两个相似的句子最终会以相同的编码结束：

* “日落来了我[凝视]墨西哥的天空不是大自然的壮丽”

使用这些标准化技术，您的模型将需要更少的训练数据并且可以更好地泛化——它不需要“日落”和“日落”的大量示例来了解它们的含义相同，并且它能够有意义 “México”，即使它在训练集中只看到“mexico”。 当然，标准化也可能会擦除一些信息，因此请始终牢记上下文：例如，如果您正在编写一个从采访文章中提取问题的模型，它肯定应该处理“？” 作为一个单独的令牌而不是丢弃它，因为它是这个特定任务的有用信号。

### 文本分割（标记化）

文本标准化后，您需要将其分解为要向量化的单元（标记），这一步骤称为“标记化”。 您可以通过三种不同的方式执行此操作：
* 单词级标记化，其中标记是空格分隔（或标点分隔）的子字符串。 一种变体是在适用时进一步将单词拆分为子词，例如将“凝视”视为“star+ing”或“call”视为“call+ed”。
* N-gram 标记化，其中标记是 N 个连续单词的组。 例如，“the cat”或“he was”将是 2-gram 标记（也称为 bigrams）。
* 字符级标记化，其中每个字符都是它自己的标记。 在实践中，这种方案很少使用，你只能在专门的上下文中看到它，比如文本生成或语音识别。

通常，您将始终使用词级或 N-gram 标记化。 有两种文本处理模型：一种关心词序，称为“序列模型”，另一种将输入词视为一个集合，丢弃其原始顺序，称为“词袋模型”。 如果您正在构建一个序列模型，您将使用词级标记化，如果您正在构建一个词袋模型，您将使用 N-gram 标记化：N-grams 是一种人工注入的方法 少量的局部词序信息进入模型。 在本章中，您将详细了解每种类型的模型以及何时使用它们。

> **理解 N-gram 和词袋**
>
> 单词 N-gram 是可以从句子中提取的 N 个（或更少）连续单词的组。 相同的概念也可以应用于字符而不是单词。
> 
> 这是一个简单的例子。 考虑句子“the cat sat on the mat”。 它可以分解为以下一组 2-gram：
> ```
> {"the", "the cat", "cat", "cat sat", "sat",
>  "sat on", "on", "on the", "the mat", "mat"}
> ```
> 
> 它也可以分解为以下一组 3-gram：
> ```
> {"the", "the cat", "cat", "cat sat", "the cat sat",
>  "sat", "sat on", "on", "cat sat on", "on the",
>  "sat on the", "the mat", "mat", "on the mat"}
> ```
> 
> 这样的集合分别称为 bag-of-2-grams bag-of-3-grams 或 。这里的术语包指的是您正在处理一组标记而不是列表或序列：标记没有特定的顺序。这一系列标记化方法称为词袋袋N-grams（或）。
> 
> 因为bag-of-words不是保序的tokenization方法（生成的token被理解为一个集合，而不是一个序列，丢失了句子的一般结构），> 所以它倾向于用于浅层语言处理模型而不是深度学习模型。提取 N-gram 是特征工程的一种形式，深度学习序列模型摒弃了这种手动方法，取而代之的是分层特征学习。通过查看连续的单词或字符序列，一维 convnets、循环神经网络和 Transformer 能够学习单词和字符组的表示，而无需明确告知此类组的存在。

### 词汇索引

将文本拆分为标记后，您需要将每个标记编码为数字表示。 您可能会以无状态的方式执行此操作，例如，通过将每个标记散列到一个固定的二进制向量中——但实际上，您要做的方法是构建训练数据中找到的所有术语的索引（ “词汇表”），并为词汇表中的每个条目分配一个唯一的整数。

像这样的东西：

In [ ]:
vocabulary = {}
for text in dataset:
    text = standardize(text)
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)

然后，您可以将该整数转换为可由神经网络处理的向量编码，例如单热向量：

In [4]:
def one_hot_encode_token(token):
    vector = np.zeros((len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1
    return vector

time: 746 µs (started: 2021-08-01 22:55:45 +08:00)


请注意，在此步骤中，通常会将词汇表限制为仅在训练数据中找到的前 20,000 个或 30,000 个最常见的单词。 任何文本数据集都倾向于包含大量独特的术语，其中大部分只出现一两次——索引这些稀有术语会导致特征空间过大，其中大多数特征几乎没有信息内容。

还记得你在第 4 章和第 5 章中在 IMDB 数据集上训练你的第一个深度学习模型吗？ 您使用的数据来自 keras.datasets.imdb，已经被预处理成整数序列，其中每个整数代表一个给定的单词。 当时，我们使用设置 num_words=10000，以将我们的词汇量限制在训练数据中最常见的前 10,000 个单词。

现在，这里有一个我们不应忽视的重要细节：当我们在词汇索引中查找新标记时，它可能不一定存在。 您的训练数据可能不包含“cherimoya”一词的任何实例（或者您可能因为它太罕见而将其从索引中排除），因此执行 token_index =vocabulary["cherimoya"] 可能会导致 KeyError。 为了解决这个问题，您应该使用“词汇外”索引（缩写为 OOV 索引），它是所有不在索引中的标记的统称。 它通常是索引 1：你实际上是在做 token_index =vocabulary.get(token, 1)。 将整数序列解码回单词时，您将 1 替换为“[UNK]”（您将其称为“OOV 标记”）之类的内容。

“为什么使用 1 而不是 0？”，您可能会问。 那是因为 0 已经被占用了。 您通常会使用两种特殊标记：OOV 标记（索引 1）和“掩码标记”（索引 0）。 虽然 OOV 令牌的意思是“这是一个我们不认识的词”，但掩码令牌告诉我们“忽略我，我不是一个词”。 你会特别用它来填充序列数据：因为数据批次需要是连续的，一批序列数据中的所有序列必须具有相同的长度，所以较短的序列应该填充到最长序列的长度。 如果你想用序列 [5, 7, 124, 4, 89] 和 [8, 34, 21] 制作一批数据，它必须是这样的：
```
[[5, 7, 124, 4, 89]
 [8, 34, 21, 0, 0]]
```

您在第 4 章和第 5 章中使用过的 IMDB 数据集整数序列批次以这种方式填充了零。

### 使用 `TextVectorization` 层

到目前为止，我们介绍的每一步都可以很容易地在纯 Python 中实现。 也许你可以这样写：

In [5]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)
    
    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()
    
    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict((v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)
    
vectorizer = Vectorizer()

dataset = ["I write, erase, rewrite",
           "Erase again, and then",
           "A poppy blooms.",
          ]

vectorizer.make_vocabulary(dataset)

time: 1.66 ms (started: 2021-08-01 23:15:27 +08:00)


它可以完成以下工作：

In [6]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
encoded_sentence

[2, 3, 5, 7, 1, 5, 6]

time: 13.2 ms (started: 2021-08-01 23:16:11 +08:00)


In [7]:
decoded_sentence = vectorizer.decode(encoded_sentence)
decoded_sentence

'i write rewrite and [UNK] rewrite again'

time: 4.49 ms (started: 2021-08-01 23:18:50 +08:00)


然而，使用这样的东西不会有很好的性能。 在实践中，您将使用 Keras TextVectorization 层，它快速高效，可以直接放入 tf.data 管道或 Keras 模型中。

这是 TextVectorization 层的样子：

In [6]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# 配置层以返回编码为整数索引的单词序列。 还有其他几种可用的输出模式，我们将在稍后看到。
text_vectorization = TextVectorization( output_mode="int", )

time: 21.8 ms (started: 2021-08-03 08:18:56 +08:00)


默认情况下，TextVectorization 层将使用设置“转换为小写并删除标点符号”进行文本标准化，并使用“拆分空白”进行标记化。 但重要的是，您可以为标准化和标记化提供自定义函数，这意味着该层足够灵活以处理任何用例。 请注意，此类自定义函数应该对张量进行操作，而不是常规 Python 字符串！ 例如，默认层 tf.string 行为等效于以下内容：

In [6]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
#     将字符串转换为小写
    lowercase_string = tf.strings.lower(string_tensor) 
#     用空字符串替换标点符号
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
#     在空白处拆分字符串
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(output_mode="int",
                                       standardize=custom_standardization_fn,
                                       split=custom_split_fn,)

time: 18.6 ms (started: 2021-08-01 23:31:05 +08:00)


要索引文本语料库的词汇表，只需使用生成字符串的 Dataset 对象或仅使用 Python 字符串列表调用图层的 adapt() 方法：

In [7]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",]

text_vectorization.adapt(dataset)

time: 1.17 s (started: 2021-08-01 23:33:29 +08:00)


请注意，您可以通过 get_vocabulary() 检索计算出的词汇表——如果您需要将编码为整数序列的文本转换回单词，这会很有用。 词汇表中的前两个条目是掩码标记（索引 0）和 OOV 标记（索引 1）。 词汇表中的条目按频率排序——因此对于真实世界的数据集，诸如“the”或“a”之类的非常常见的词将排在第一位。

> 清单 11.1 显示词汇表

In [12]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

time: 22.5 ms (started: 2021-08-01 23:35:14 +08:00)


为了演示，让我们尝试编码然后解码一个例句：

In [14]:
vocabulary = text_vectorization.get_vocabulary()

time: 2.65 ms (started: 2021-08-01 23:36:19 +08:00)


In [15]:
test_sentence = "I write, rewrite, and still rewrite again"

time: 734 µs (started: 2021-08-01 23:36:28 +08:00)


In [17]:
encoded_sentence = text_vectorization(test_sentence)
encoded_sentence

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 7,  3,  5,  9,  1,  5, 10])>

time: 59.3 ms (started: 2021-08-01 23:36:58 +08:00)


In [19]:
inverse_vocab = dict(enumerate(vocabulary))
inverse_vocab

{0: '',
 1: '[UNK]',
 2: 'erase',
 3: 'write',
 4: 'then',
 5: 'rewrite',
 6: 'poppy',
 7: 'i',
 8: 'blooms',
 9: 'and',
 10: 'again',
 11: 'a'}

time: 6.61 ms (started: 2021-08-01 23:37:26 +08:00)


In [20]:
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
decoded_sentence  

'i write rewrite and [UNK] rewrite again'

time: 52.8 ms (started: 2021-08-01 23:37:53 +08:00)


在 tf.data 管道中或作为模型的一部分使用 TextVectorization 层重要的是，因为 TextVectorization 主要是一个字典查找操作，它不能在 GPU（或 TPU）上执行——只能在 CPU 上执行。 因此，如果您在 GPU 上训练模型，那么 TextVectorization 层将在 CPU 上运行，然后将其输出发送到 GPU。 这具有重要的性能影响。

我们可以通过两种方式使用我们的 TextVectorization 层。 第一个选项是将其放入管道中，如下所示：tf.data

> 清单 11.2 选项 1：tf.data 管道中的 TextVectorization

In [ ]:
int_sequence_dataset = string_dataset.map(text_vectorization)

第二种选择是让它成为模型的一部分（毕竟它是一个 Keras 层），就像这样：

> 清单 11.3 选项 2：模型中的 TextVectorization

In [ ]:
# 创建一个需要字符串的符号输入。
text_input = keras.Input(shape=(), dtype="string") 

# 对其应用文本矢量化层。
vectorized_text = text_vectorization(text_input) 

# 您可以继续在顶部链接新层——只是您的常规功能 API 模型。
embedded_input = keras.layers.Embedding(...)(vectorized_text) 
output = ... 

model = keras.Model(text_input, output)

> 两者之间有一个重要区别：如果矢量化步骤是模型的一部分，它将与模型的其余部分同步发生。 这意味着在每个训练步骤中，模型的其余部分（放置在 GPU 上）必须等待 TextVectorization 层（放置在 CPU 上）的输出准备就绪才能开始工作（见图 TODO） . 同时，将该层放入 tf.data 管道中，您可以在 CPU 上对数据进行异步预处理：当 GPU 在一批矢量化数据上运行模型时，CPU 通过矢量化下一批原始字符串来保持忙碌。
> 
> 因此，如果您在 GPU 或 TPU 上训练模型，您可能希望使用选项 1 以获得最佳性能。 这就是我们将在本章中的所有实际示例中执行的操作。 但是，在 CPU 上进行训练时，同步处理很好：无论您选择选项 1 还是选项 2，您都将获得 100% 的内核利用率。
>
> 现在，如果您要将我们的模型导出到生产环境，您可能希望发布一个接受原始字符串作为输入的模型，就像上面选项二的代码片段一样——否则，您将不得不重新实现文本标准化和标记化 您的生产环境（可能在 JavaScript 中？），并且您将面临引入小的预处理差异的风险，这会损害模型的准确性。 值得庆幸的是，TextVectorization 层使您能够将文本预处理直接包含到您的模型中，从而使其更易于部署——即使您最初将该层用作管道的一部分。 在 TODO 部分，您将学习如何将 tf.data 导出为包含文本预处理的仅推理训练模型。

到目前为止，您已经了解了有关文本预处理的所有知识——让我们进入建模阶段。

## 表示词组的两种方法：集合和序列

机器学习模型应该如何表示单个单词是一个相对没有争议的问题：它们是分类特征（来自预定义集合的值），我们知道如何处理这些特征。 它们应该被编码为特征空间中的维度，或作为类别向量（在这种情况下是词向量）。 然而，一个更成问题的问题是如何对单词编织成句子的方式进行编码：词序。

自然语言中的顺序问题是一个有趣的问题：与时间序列的步骤不同，句子中的单词没有自然的规范顺序。 不同的语言以非常不同的方式对相似的单词进行排序。 例如，英语的句子结构与日语的句子结构大不相同。 即使在给定的语言中，您通常也可以通过稍微重新排列单词以不同的方式说出相同的内容。 更进一步，如果你将一个短句中的单词完全随机化，你仍然可以在很大程度上弄清楚它在说什么——尽管在许多情况下似乎会出现明显的歧义。 秩序显然很重要，但它与意义的关系并不简单。

如何表示词序是不同类型 NLP 架构产生的关键问题。 您可以做的最简单的事情就是丢弃顺序并将文本视为一组无序的单词——这为您提供了词袋模型。 您还可以决定严格按照单词出现的顺序处理单词，一次一个，就像时间序列中的步骤一样——然后您可以利用上一章中的循环模型。 最后，混合方法也是可能的：Transformer 架构在技术上与顺序无关，但它将词位置信息注入它处理的表示中，这使其能够同时查看句子的不同部分（与 RNN 不同），同时仍然 订单意识。 因为它们考虑了词序，所以 RNN 和 Transformer 都被称为序列模型。

从历史上看，机器学习在 NLP 中的大多数早期应用只涉及词袋模型。 随着循环神经网络的重生，对序列模型的兴趣在 2015 年才开始上升。 今天，这两种方法仍然适用。 让我们看看它们是如何工作的，以及何时利用它们。

我们将在一个著名的文本分类基准上演示每种方法：IMDB 电影评论情感分类数据集。 在第 4 章和第 5 章中，您已经使用了 IMDB 数据集的预矢量化版本——现在，让我们处理原始 IMDB 文本数据，就像您在现实世界中处理新的文本分类问题时所做的那样。

### 准备 IMDB 电影评论数据

让我们从 Andrew Maas 的斯坦福页面下载数据集开始，并解压缩它：

In [21]:
%%bash

cd data

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz && tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   315k      0  0:04:20  0:04:20 --:--:--  321k


time: 6min 31s (started: 2021-08-01 23:53:57 +08:00)


剩下一个名为 aclImdb 的目录，其结构如下：
```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

In [23]:
%%bash

cd data/aclImdb/

time: 99.9 ms (started: 2021-08-02 00:06:29 +08:00)


例如，train/pos/ 目录包含一组 12,500 个文本文件，每个文件都包含用作训练数据的正面情感电影评论的文本正文。 负面情绪评论存在于neg目录中。 总共有 25,000 个文本文件用于负训练，另外 25,000 个用于测试。

那里还有一个 train/unsup 子目录，我们不需要它。 让我们删除它：

In [26]:
![ -d data/aclImdb/train/unsup ] && rm -r data/aclImdb/train/unsup

time: 236 ms (started: 2021-08-02 00:07:50 +08:00)


看看其中一些文本文件的内容。 无论您是处理文本数据还是图像数据，请记住在深入建模之前始终检查数据的外观。 它将为您的模型实际做什么奠定您的直觉。

In [28]:
!cat data/aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoytime: 184 ms (started: 2021-08-02 00:08:32 +08:00)


接下来，让我们通过在新目录 aclImdb/val 中设置 20% 的训练文本文件来准备验证集：

In [107]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("data/aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category, exist_ok=True)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files)) 
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir/category/fname, val_dir/category/fname)

time: 3.24 s (started: 2021-08-02 00:58:44 +08:00)


还记得在第 8 章中，我们如何使用实用程序 image_dataset_from_directory 为目录结构创建一批图像及其标签吗？ 您可以使用实用程序 text_dataset_from_directory 对文本文件执行完全相同的数据集操作。 让我们创建三个 Dataset 对象，用于训练、验证和测试：

In [4]:
from tensorflow import keras

batch_size = 32

train_ds = keras.preprocessing.text_dataset_from_directory( 
    "data/aclImdb/train", batch_size=batch_size)

val_ds = keras.preprocessing.text_dataset_from_directory(
    "data/aclImdb/val", batch_size=batch_size)

test_ds = keras.preprocessing.text_dataset_from_directory(
    "data/aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
time: 4.99 s (started: 2021-08-04 14:14:27 +08:00)


运行这一行应该输出“Found 20000 files分为2个类”； 如果您看到“发现 70000 个文件属于 3 个类”，则表示您忘记删除 aclImdb/train/unsup 目录。

这些数据集产生的输入是 TensorFlow 张量，目标是编码值“0”或“1”的 tf.string int32 张量：

> 清单 11.4 显示第一批的形状和数据类型

In [5]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Ah, the best and funniest movie about female football fans, only slightly better than the 1982 saga of teenage delusion set in North London (qv). By the way, I just watched this on Film 4 [2008-12-21] and am ruing my inability to set the PVR).<br /><br />This is easily my second favourite football movie after "Mike Bassett: England Manager", but this time with the added twist of looking like a guerrilla piece of movie-making from a team who apparently keep making movies which are banned in the country in which they are made (just think about the bit where the girls are taken from the stadium just as the Sun is setting: fast reactions all round). <br /><br />It is rare for a movie to make me laugh out loud, but when the rural soldier escorted one of the girls into the lavvies while forcing her to wear an inpromptu mask made from a poster of Ali Daei, I couldn\'

可以了，好了。 现在让我们尝试从这些数据中学习一些东西。

### 将单词作为集合处理：词袋方法

对一段文本进行编码以供机器学习模型处理的最简单方法是丢弃顺序并将其视为一组（“袋子”）标记。 您可以查看单个单词（unigrams），也可以通过查看连续标记组（N-grams）来尝试恢复一些本地顺序信息。

**二进制编码的单字（UNIGRAMS）**

如果用一袋单字，句子“the cat sat on the mat”就变成了：
```Python
{"cat", "mat", "on", "sat", "the"}
```

这种编码的主要优点是您可以将整个文本表示为单个向量，其中每个条目都是给定单词的存在指示符。 例如，使用二进制编码（multi-hot），您可以将文本编码为一个向量，其维度与词汇表中的单词数量一样多——0 几乎无处不在，一些 1 表示对文本中存在的单词进行编码的维度 . 这就是我们在第 4 章和第 5 章中处理文本数据时所做的。

让我们在我们的任务中试试这个。

首先，让我们用 TextVectorization 层准备处理我们的原始文本数据集，以便它们产生二进制词向量。 我们的层只会查看单个单词（也就是说，unigrams）。

> 清单 11.5 使用 TextVectorization 层预处理我们的数据集

In [7]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# 将词汇量限制为 20,000 个最常用的单词。 否则，我们将索引训练数据中的每个单词——可能有数万个只出现一次或两次的术语，
# 因此没有信息量。 一般来说，20,000 是适合文本分类的词汇量。
text_vectorization = TextVectorization(max_tokens=20000, 
                                       output_mode="binary", # 将输出标记编码为二进制向量。
                                      )

# 准备一个仅产生原始文本输入（无标签）的数据集。
text_only_train_ds = train_ds.map(lambda x, y: x) 
# 通过adapter() 方法，使用该数据集来索引数据集词汇表。
text_vectorization.adapt(text_only_train_ds) 

# 准备我们的训练、验证和测试数据集的处理版本。
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y)) 
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

time: 5.71 s (started: 2021-08-04 14:17:58 +08:00)


您可以尝试检查以下数据集之一的输出：

In [34]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape) #输入是 20,000 维向量的批次。
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0]) #这些向量完全由 1 和 0 组成。
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)
time: 272 ms (started: 2021-08-03 14:55:55 +08:00)


接下来，让我们编写一个可重用的模型构建函数，我们将在本节的所有实验中使用该函数。

> 清单 11.7 我们的模型构建工具

In [35]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

time: 935 µs (started: 2021-08-03 15:20:32 +08:00)


In [36]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense (Dense)                (None, 16)                320016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
time: 96.8 ms (started: 2021-08-03 15:20:53 +08:00)


最后，让我们训练和测试我们的模型：

In [37]:
# model = get_model()
# model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)]

# 我们调用数据集将它们缓存在内存中：这样，在第一个 epoch 期间，我们只会 cache() 进行一次预处理，
# 并且我们将在接下来的 epoch 中重用预处理过的文本。 只有当数据小到可以放入内存时才能这样做。
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 20s 24ms/step - loss: 0.3924 - accuracy: 0.8360 - val_loss: 0.2779 - val_accuracy: 0.8872
Epoch 2/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2668 - accuracy: 0.9011 - val_loss: 0.2778 - val_accuracy: 0.8972
Epoch 3/10
625/625 [==============================] - 9s 14ms/step - loss: 0.2391 - accuracy: 0.9179 - val_loss: 0.2890 - val_accuracy: 0.8960
Epoch 4/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2266 - accuracy: 0.9226 - val_loss: 0.3013 - val_accuracy: 0.8968
Epoch 5/10
625/625 [==============================] - 9s 15ms/step - loss: 0.2187 - accuracy: 0.9302 - val_loss: 0.3165 - val_accuracy: 0.8974
Epoch 6/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2143 - accuracy: 0.9322 - val_loss: 0.3227 - val_accuracy: 0.8966
Epoch 7/10
625/625 [==============================] - 8s 14ms/step - loss: 0.2097 - accuracy: 0.9334 - val_loss: 0.3353 - val_accuracy: 0.

time: 1min 47s (started: 2021-08-03 15:23:21 +08:00)


In [38]:
model = keras.models.load_model("binary_1gram.keras")

print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 13s 17ms/step - loss: 0.2891 - accuracy: 0.8892
Test acc: 0.889
time: 13.7 s (started: 2021-08-03 15:26:27 +08:00)


这使我们的测试准确率为 89.2%：不错！ 请注意，在这种情况下，由于数据集是一个平衡的二分类数据集（正样本与负样本一样多），我们可以在不训练实际模型的情况下达到的“朴素基线”只有 50%。 同时，在不利用外部数据的情况下，可以在该数据集上获得的最佳分数是大约 95% 的测试准确率。

**二进制编码的 BIGRAMS**

当然，放弃词序是非常还原的，因为即使是原子概念也可以通过多个词来表达：“美国”一词传达的概念与将“国家”和“联合”这两个词分开的含义截然不同。 出于这个原因，您通常最终会通过查看 N-gram 而不是单个单词（最常见的是二元组）将本地顺序信息重新注入到您的词袋表示中。

使用 bigrams，我们的句子变成：
```
{"the", "the cat", "cat", "cat sat", "sat",
 "sat on", "on", "on the", "the mat", "mat"}
 ```

TextVectorization 层可以配置为返回任意 N-grams：bigrams、trigrams 等。只需传递一个参数，如下所示：

> 清单 11.9 配置 TextVectorization 层以返回二元组

In [39]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="binary",
)

time: 19.2 ms (started: 2021-08-03 15:28:52 +08:00)


让我们测试您的模型在此类二进制编码的二元组上训练时的表现：

> 清单 11.10 训练和测试二进制二元模型

In [41]:
text_vectorization.adapt(text_only_train_ds)
                         
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

model = get_model()
model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                             save_best_only=True)]
                         
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 20s 31ms/step - loss: 0.3805 - accuracy: 0.8431 - val_loss: 0.2672 - val_accuracy: 0.9006
Epoch 2/10
625/625 [==============================] - 10s 16ms/step - loss: 0.2398 - accuracy: 0.9162 - val

time: 2min 22s (started: 2021-08-03 15:30:40 +08:00)


In [42]:
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 19s 23ms/step - loss: 0.2653 - accuracy: 0.8973
Test acc: 0.897
time: 19.7 s (started: 2021-08-03 15:33:16 +08:00)


我们现在获得了 90.4% 的测试准确率，这是一个显着的改进！ 事实证明局域顺序非常重要。

**带有 TF-IDF 编码的 BIGRAMS**

您还可以通过计算每个单词或 N-gram 出现的次数来为该表示添加更多信息，也就是说，通过获取文本上的单词的直方图：
```
{"the": 2, "the cat": 1, "cat": 1, "cat sat": 1, "sat": 1,
 "sat on": 1, "on": 1, "on the": 1, "the mat: 1", "mat": 1}
```

如果您在进行文本分类，那么了解某个单词在样本中出现的次数至关重要：无论情绪如何，任何足够长的电影评论都可能包含“terrible”一词，但评论中包含“terrible”一词的许多实例 很可能是负面的。

下面是如何使用 TextVectorization 层计算双字组出现的次数：

> 清单 11.11 配置 TextVectorization 层以返回标记计数

In [43]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

time: 5.51 ms (started: 2021-08-03 15:36:20 +08:00)


现在，当然，无论您在看什么电影评论，有些词肯定会比其他词更频繁地出现。文本是关于什么的。单词“the”、“a”、“is”和“are”将始终主导您的字数直方图，淹没其他单词——尽管在分类上下文中几乎是无用的特征。我们如何解决这个问题？

您已经猜到了：通过规范化。我们可以通过减去均值并除以方差（在整个训练数据集中计算）来标准化字数。那是有道理的。除了……大多数向量化句子几乎完全由零组成（我们上面的示例具有 12 个非零条目和 19,988 个零条目），这是一种称为“稀疏性”的属性。这是一个很好的属性，因为它可以显着减少计算负载并降低过度拟合的风险。如果我们从每个特征中减去平均值，我们就会破坏稀疏性。因此，我们使用的任何归一化方案都应该是只除法。那么，我们应该使用什么作为分母呢？最佳实践是使用称为 TF-IDF 规范化的东西——TF-IDF 代表“词频，逆文档频率”。

> **理解 TF-IDF 归一化**
> 
> 给定术语在文档中出现的次数越多，该术语对于理解文档的内容就越重要。 同时，术语在数据集中所有文档中出现的频率也很重要：出现在几乎每个文档中的术语（如“the”或“a”）并不是特别有用，而仅出现在 所有文本的一小部分（如“Herzog”）非常独特，因此很重要。 TF-IDF 是一个融合了这两个想法的指标。 它通过获取“术语频率”、该术语在当前文档中出现的次数并将其除以“文档频率”的度量来对给定术语进行加权，从而估计该术语在数据集中出现的频率。 您可以将其计算为：
>
> ```
def tfidf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
    return term_freq / doc_freq
```

TF-IDF 非常常见，以至于它内置于 TextVectorization 层中。 开始使用它所需要做的就是将 output_mode 参数切换为：

> 清单 11.12 配置 TextVectorization 层以返回 TF-IDF 加权输出

In [45]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf-idf",
)

time: 26.1 ms (started: 2021-08-03 15:42:00 +08:00)


> 清单 11.13 训练和测试 TF-IDF 双元组模型

In [46]:
text_vectorization.adapt(text_only_train_ds) 

tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

model = get_model()
model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                             save_best_only=True)]

model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 23s 36ms/step - loss: 0.4763 - accuracy: 0.7936 - val_loss: 0.3335 - val_accuracy: 0.8828
Epoch 2/10
625/625 [==============================] - 10s 15ms/step - loss: 0.2987 - accuracy: 0.8828 - val

time: 2min 30s (started: 2021-08-03 15:42:56 +08:00)


In [ ]:
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

这让我们在 IMDB 分类任务上获得了 89.8% 的测试准确率：在这种情况下它似乎不是特别有用。 但是，对于许多文本分类数据集，与纯二进制编码相比，使用 TF-IDF 时通常会看到一个百分点的增加。

> **导出处理原始字符串的模型**
>
> 在上面的示例中，我们将文本标准化、拆分和索引作为管道的一部分。 但是如果我们想导出一个独立于这个管道的独立模型 tf.data，我们应该确保它包含自己的文本预处理（否则，你必须在生产环境中重新实现，这可能具有挑战性或可能导致 训练数据和生产数据之间的细微差异）。 谢天谢地，这很容易。
> 
> 只需创建一个新模型，重用您的 TextVectorization 层，并将您刚刚训练的模型添加到其中：
>```python
inputs = keras.Input(shape=(1,), dtype="string") 
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)
>```
>

> 生成的模型可以处理成批的原始字符串：
>
> ```
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([["That was an excellent movie, I loved it."],])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")
```

### 将单词作为序列处理：序列模型方法

过去的几个例子清楚地表明词序很重要：基于顺序的特征的手动工程，例如二元组，产生了很好的准确度提升。 现在，请记住：深度学习的历史是从手动特征工程转向让模型仅通过接触数据来学习自己的特征。 如果我们不是手动制作基于顺序的特征，而是将模型暴露给原始单词序列，然后让它自己找出这些特征，会怎么样？ 这就是序列模型的意义所在。

要实现序列模型，您首先要将输入样本表示为整数索引序列（一个整数代表一个单词）。 然后，您将每个整数映射到一个向量，以获得向量序列。 最后，您将这些向量序列输入到一个层堆栈中，这些层可以将来自相邻向量的特征相互关联，例如 1D 卷积网络、RNN 或 Transformer。

在 2016-2017 年左右的一段时间里，双向 RNN（特别是双向 LSTM）被认为是序列建模的最新技术。 由于您已经熟悉这种架构，这就是我们将在下面的第一个序列模型示例中使用的架构。 然而，如今，序列建模几乎普遍使用 Transformer 完成，我们将在稍后介绍。 奇怪的是，一维卷积网络在 NLP 中从来都不是很流行——尽管根据我自己的经验，深度可分离的一维卷积的残余堆栈通常可以实现与双向 LSTM 相当的性能，同时大大降低了计算成本。

**第一个实际例子**

让我们在实践中尝试第一个序列模型。 首先，让我们准备返回整数序列的数据集。

> 清单 11.14 准备整数序列数据集

In [8]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# 为了保持可管理的输入大小，我们将在前 600 个单词之后截断输入。 
# 这是一个合理的选择，因为平均评论长度为 233 字，只有 5% 的评论超过 600 字形成连续批次。
max_length = 600
max_tokens = 20000

text_vectorization = TextVectorization(max_tokens=max_tokens,
                                       output_mode="int",
                                       output_sequence_length=max_length, )

# 准备一个仅产生原始文本输入（无标签）的数据集。
text_only_train_ds = train_ds.map(lambda x, y: x) 
# 通过adapter() 方法，使用该数据集来索引数据集词汇表。
text_vectorization.adapt(text_only_train_ds) 

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

time: 7.61 s (started: 2021-08-04 14:18:16 +08:00)


接下来，让我们制作一个模型。 我们可以使用的将整数序列转换为向量序列的最简单工具是对整数进行单热编码（每个维度代表词汇表中一个可能的术语）。 在这些 one-hot 向量之上，我们将添加一个简单的双向 LSTM。

> 清单 11.15 建立在 one-hot 编码向量序列之上的序列模型

In [9]:
import tensorflow as tf
from tensorflow.keras import layers
 
# 一个输入是 100 个整数的序列。
inputs = keras.Input(shape=(None,), dtype="int64") 

# 将整数编码为 20,000 维的二进制向量。
embedded = tf.one_hot(inputs, depth=max_tokens)

# 添加双向 LSTM。
x = layers.Bidirectional(layers.LSTM(32))(embedded) 
x = layers.Dropout(0.5)(x)
# 最后，添加一个分类层。
outputs = layers.Dense(1, activation="sigmoid")(x) 

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
tf.one_hot (TFOpLambda)      (None, None, 20000)       0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                5128448   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
_________________________________________________________________
time: 1.36 s (started: 2021-08-04 14:18:27 +08:00)


现在，让我们训练我们的模型。

> 清单 11.16 训练第一个基本序列模型

In [10]:
callbacks = [keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                             save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 122s 177ms/step - loss: 0.5218 - accuracy: 0.7531 - val_loss: 0.3555 - val_accuracy: 0.8580
Epoch 2/10
625/625 [==============================] - 108s 173ms/step - loss: 0.3560 - accuracy: 0.8723 - val_loss: 0.3107 - val_accuracy: 0.8744
Epoch 3/10
625/625 [==============================] - 109s 174ms/step - loss: 0.2767 - accuracy: 0.9032 - val_loss: 0.2732 - val_accuracy: 0.8908
Epoch 4/10
625/625 [==============================] - 109s 175ms/step - loss: 0.2305 - accuracy: 0.9218 - val_loss: 0.3045 - val_accuracy: 0.8784
Epoch 5/10
625/625 [==============================] - 108s 172ms/step - loss: 0.2115 - accuracy: 0.9280 - val_loss: 0.3783 - val_accuracy: 0.8292
Epoch 6/10
625/625 [==============================] - 110s 176ms/step - loss: 0.1820 - accuracy: 0.9385 - val_loss: 0.3038 - val_accuracy: 0.8900
Epoch 7/10
625/625 [==============================] - 109s 175ms/step - loss: 0.1637 - accuracy: 0.9459 - val_loss: 0.3050 -

time: 18min 25s (started: 2021-08-04 14:32:40 +08:00)


In [ ]:
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

第一个观察：这个模型训练非常慢，尤其是与上一节的轻量级模型相比。 这是因为我们的输入非常大：每个输入样本都被编码为一个大小为 (600, 20000) 的矩阵（每个样本 600 个单词，20000 个可能的单词）。 对于单个电影评论来说，这是 12,000,000 个浮点数。 我们的双向 LSTM 有很多工作要做。 其次，该模型只能达到 87% 的测试准确率——它的表现几乎不如我们的（非常快的）二元一元模型。

显然，使用 one-hot 编码将单词转换为向量，这是我们能做的最简单的事情，并不是一个好主意。 有一个更好的方法：词嵌入。

**理解词嵌入**

至关重要的是，当您通过 one-hot 编码对某些内容进行编码时，您正在做出特征工程决策。您正在向模型中注入关于特征空间结构的基本假设。该假设是您编码的不同标记都彼此独立：实际上，one-hot 向量都彼此正交。就文字而言，这种假设显然是错误的。单词形成一个结构化的空间：它们彼此共享信息。在大多数句子中，“电影”和“电影”这两个词可以互换，因此表示“电影”的向量不应与表示“电影”的向量正交——它们应该是相同的向量，或者足够接近。

为了更抽象一点，两个词向量之间的几何关系应该反映这些词之间的语义关系。例如，在一个合理的词向量空间中，你会期望同义词被嵌入到相似的词向量中，一般来说，你会期望任意两个词向量之间的几何距离（例如余弦距离或 L2 距离）与相关词之间的“语义距离”。表示不同事物的词应该彼此远离，而相关词应该靠得更近。

词嵌入是词的向量表示，可以实现这一点：它们将人类语言映射到结构化的几何空间中。

虽然通过 one-hot 编码获得的向量是二进制的、稀疏的（主要由零组成）和非常高维的（与词汇表中的单词数量相同），但词嵌入是低维浮点向量（即 是密集向量，而不是稀疏向量）； 见图 11.2。 在处理非常大的词汇表时，通常会看到 256 维、512 维或 1,024 维的词嵌入。 另一方面，one-hot 编码词通常会导致 20,000 维或更大的向量（在这种情况下捕获 20,000 个标记的词汇表）。 因此，词嵌入将更多信息打包到更少的维度中。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4r73737dj30my0rgmys.jpg)

除了表示之外，词嵌入也是表示，它们的结构是从数据中学习到的密集结构。 相似的词被嵌入在附近的位置，而且嵌入空间中的特定词是有意义的。 为了更清楚地说明这一点，让我们看一个具体的方向示例。

在图 11.3 中，四个词嵌入在 2D 平面上：cat、dog、wofl 和 Tiger。 使用我们在这里选择的向量表示，这些词之间的一些语义关系可以编码为几何变换。 例如，同一个向量允许我们从猫到老虎，从狗到狼：这个向量可以解释为“从宠物到野生动物”的向量。 类似地，另一个向量让我们从狗到猫，从狼到老虎，这可以解释为“从犬到猫”的向量。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4r9zfarpj30xi0lygmb.jpg)

在现实世界的词嵌入空间中，有意义的几何变换的常见示例是“性别”向量和“复数”向量。 例如，通过向向量“king”添加“female”向量，我们得到向量“queen”。 通过添加“复数”向量，我们得到“国王”。 词嵌入空间通常具有数千个此类可解释且可能有用的向量。

让我们看看如何在实践中使用这样的嵌入空间。 有两种方法可以获得词嵌入：
* 与您关心的主要任务（例如文档分类或情感预测）一起学习词嵌入。 在此设置中，您从随机词向量开始，然后以与学习神经网络权重相同的方式学习词向量。
* 将使用与您尝试解决的任务不同的机器学习任务预先计算的词嵌入加载到您的模型中。 这些被称为预训练词嵌入。

让我们回顾一下这些方法中的每一种。

**使用Embedding层学习词嵌入**

是否有一些理想的词嵌入空间可以完美地映射人类语言并可以用于任何自然语言处理任务？ 可能，但我们还没有计算出任何类似的东西。 此外，没有人类语言这样的东西——有许多不同的语言，它们彼此并不同构，因为一种语言是特定文化和特定语境的反映。 但更实用的是，什么是一个好的词嵌入空间在很大程度上取决于你的任务：英语电影评论情感分析模型的完美词嵌入空间可能与英语法律的完美嵌入空间不同。 文档分类模型，因为某些语义关系的重要性因任务而异。

因此，为每个新任务学习一个新的嵌入空间是合理的。 幸运的是，反向传播使这变得容易，而 Keras 使这变得更加容易。 这是关于学习层的权重：Embedding层。

> 清单 11.17 实例化一个层

In [11]:
# 嵌入层至少有两个参数：可能的标记数量和嵌入的维度（这里是 256）。
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

time: 7.94 ms (started: 2021-08-04 15:02:49 +08:00)


该层最好理解为将整数索引（代表特定单词的嵌入）映射到密集向量的字典。 它接受整数作为输入，在内部字典中查找这些整数，并返回相关的向量。 它实际上是一个字典查找（见图 11.4）

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4rui2pgkj31fw052t9c.jpg)

嵌入层将形状为 (batch_size, sequence_length) 的二维整数张量作为输入，其中每个条目都是一个整数序列。 然后该层返回一个形状为（batch_size、sequence_length、embedding_dimensionality）的 3D 浮点张量。

当你实例化一个层时，它的权重（它的内部标记向量字典）嵌入最初是随机的，就像任何其他层一样。 在训练期间，这些词向量通过反向传播逐渐调整，将空间构造成下游模型可以利用的东西。 一旦经过充分训练，嵌入空间将显示出很多结构——一种专门用于训练模型的特定问题的结构。

让我们构建一个包含嵌入层的模型，并在我们的任务中对其进行基准测试：

> 清单 11.18 使用从头开始训练的嵌入层的模型

In [12]:
inputs = keras.Input(shape=(None,), dtype="int64")

embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                             save_best_only=True)
            ]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                73984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 105s 153ms/step - loss: 0.4758 - accuracy: 0.7828 - val_loss:

time: 15min 54s (started: 2021-08-04 15:52:10 +08:00)


In [13]:
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 35s 40ms/step - loss: 0.3335 - accuracy: 0.8675
Test acc: 0.868
time: 37.4 s (started: 2021-08-04 16:08:04 +08:00)


它的训练速度比 one-hot 模型快得多（因为 LSTM 只需处理 256 维向量而不是 20,000 维），并且其测试准确率相当（87%）。 然而，我们离我们的基本二元模型的结果还有一段距离。 部分原因很简单，模型查看的数据略少：bigram 模型处理完整评论，而我们的序列模型在 600 个单词后截断序列。

**了解padding和masking**

这里稍微影响模型性能的一件事是我们的输入序列全是零。 这来自于我们在 TextVectorization max_length（等于 600）中使用 output_sequence_length=max_length 选项：超过 600 个标记的句子被截断为 600 个标记的长度，而少于 600 个标记的句子在末尾用零填充，以便 它们可以与其他序列连接在一起以形成连续的批次。

我们使用双向 RNN：并行运行的两个 RNN 层，一个以自然顺序处理令牌，另一个以相反的方式处理相同的令牌。 以自然顺序查看标记的 RNN 将在最后一次迭代中只看到编码填充的向量——如果原始句子很短，可能会进行数百次迭代。 当 RNN 暴露于这些无意义的输入时，存储在 RNN 内部状态中的信息将逐渐消失。

我们需要一些方法来告诉 RNN 它应该跳过这些迭代。 有一个 API：*masking*。

该层能够生成与其输入数据相对应的“掩码”。 这个嵌入掩码是一个由 1 和 0（或 True/False 布尔值）组成的张量，形状为 (batch_size, sequence_length)，其中 entry mask[i, t] 指示应该跳过或不跳过样本 i 的时间步长 t（时间步长将 如果 mask[i, t] 为 0 或 False，则跳过，否则进行处理）。

默认情况下，此选项未激活 - 通过将 mask_zero=True 传递给您的嵌入层来打开它。 您可以通过 compute_mask() 方法检索掩码：

In [17]:
embedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)

time: 8.05 ms (started: 2021-08-04 16:15:13 +08:00)


In [19]:
some_input = [[4, 3, 2, 1, 0, 0, 0],
              [5, 4, 3, 2, 1, 0, 0],
              [2, 1, 0, 0, 0, 0, 0]]

time: 735 µs (started: 2021-08-04 16:16:42 +08:00)


In [21]:
mask = embedding_layer.compute_mask(some_input)
mask

<tf.Tensor: shape=(3, 7), dtype=bool, numpy=
array([[ True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False, False],
       [ True,  True, False, False, False, False, False]])>

time: 6.08 ms (started: 2021-08-04 16:17:50 +08:00)


在实践中，您几乎不需要手动管理mask。 相反，Keras 会自动将掩码与其代表的序列一起传递给能够处理它的每个层。 RNN 层将使用此掩码来跳过掩码步骤。 如果您的模型返回整个序列，那么损失函数也将使用掩码来跳过输出序列中的掩码步骤。

让我们尝试在启用掩码的情况下重新训练我们的模型：

> 清单 11.19 使用从头开始训练的嵌入层的模型，启用了掩码：

In [22]:
inputs = keras.Input(shape=(None,), dtype="int64")

embedded = layers.Embedding(input_dim=max_tokens, 
                            output_dim=256, 
                            mask_zero=True)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                73984     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 128s 181ms/step - loss: 0.3991 - accuracy: 0.8181 - val_loss:

time: 19min 9s (started: 2021-08-04 16:29:36 +08:00)


In [24]:
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 43s 49ms/step - loss: 0.2894 - accuracy: 0.8809
Test acc: 0.881
time: 49.5 s (started: 2021-08-04 16:49:24 +08:00)


这一次，我们达到了 88% 的测试准确率——这是一个很小但很明显的改进。

**使用预训练词嵌入**

有时，您可用的训练数据太少，以至于您无法单独使用数据来学习合适的特定于任务的词汇嵌入。 在这种情况下，您可以从预先计算的嵌入空间加载嵌入向量，而不是与您要解决的问题一起学习词嵌入，该嵌入空间您知道它是高度结构化的并具有有用的属性 - 捕获语言结构的通用方面。 在自然语言处理中使用预训练词嵌入背后的基本原理与在图像分类中使用预训练卷积网络非常相似：您没有足够的可用数据来自行学习真正强大的功能，但您期望获得所需的功能 相当通用——即常见的视觉特征或语义特征。 在这种情况下，重用在不同问题上学到的特征是有意义的。

这种词嵌入通常是使用词出现统计（观察哪些词在句子或文档中共同出现）计算的，使用各种技术，有些涉及神经网络，有些则没有。 Bengio 等人最初探索了以无监督方式计算的密集、低维词嵌入空间的想法。 在 2000 年代初期，但在 24 种最著名和最成功的词嵌入方案之一：Word2Vec 算法 (code.google.com/archive/p/word2vec) 发布后，它才开始在研究和行业应用中起飞 ，由 Google 的 Tomas Mikolov 于 2013 年开发。 Word2Vec 维度捕获特定的语义属性，例如性别。

您可以在 Keras 层下载和使用各种预先计算的词嵌入数据库。 Word2vec 就是其中之一。 另一种流行的方法称为词表示的全局嵌入向量（GloVe，nlp.stanford.edu/projects/glove），由斯坦福大学的研究人员于 2014 年开发。这种嵌入技术基于分解词共现统计矩阵。 它的开发人员已经为数百万个英语标记提供了预先计算的嵌入，这些标记是从维基百科数据和 Common Crawl 数据中获得的。

让我们看看如何开始在 Keras 模型中使用 GloVe 嵌入。 同样的方法适用于 Word2Vec 嵌入或任何其他词嵌入数据库。 您还将使用此示例来刷新前几段介绍的文本标记化技术：您将从原始文本开始，然后逐步向上。

下载 GloVe 词嵌入

首先，让我们下载在 2014 年英文维基百科数据集上预先计算的 GloVe 词嵌入。 这是一个 822 MB 的 zip 文件，其中包含 400,000 个单词（或非单词标记）的 100 维嵌入向量。

In [ ]:
%%bash

cd data

wget http://nlp.stanford.edu/data/glove.6B.zip && unzip -q glove.6B.zip

让我们解析解压后的文件（一个 .txt 文件）来构建一个索引，将单词（作为字符串）映射到它们的向量表示。

> 清单 11.20 解析 GloVe word-embeddings 文件

In [26]:
import numpy as np
path_to_glove_file = "data/glove.6B.100d.txt"

embeddings_index = {}

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.
time: 11.7 s (started: 2021-08-04 18:06:28 +08:00)


**在模型中加载 GloVe 嵌入**

接下来，让我们构建一个可以加载到层中的嵌入矩阵。 它必须是一个形状为 (max_words, embedding_dim) 的嵌入矩阵，其中每个条目 i 包含参考词索引中索引词的 embedding_dim 维向量（在标记化期间构建 i）。 请注意，索引 0 不应该代表任何单词或标记——它是一个占位符。

> 清单 11.21 准备 GloVe 词嵌入矩阵

In [27]:
embedding_dim = 100

# 检索我们之前的 TextVectorization 层索引的词汇表
vocabulary = text_vectorization.get_vocabulary()

# 使用它来创建从单词到它们在词汇表中的索引的映射
word_index = dict(zip(vocabulary, range(len(vocabulary)))) 

# 准备一个矩阵，我们将用 GloVe 向量填充
embedding_matrix = np.zeros((max_tokens, embedding_dim)) 
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
        
#     用索引 i 的词向量填充矩阵中的条目 i 。 在嵌入索引中未找到的单词将全部为零。
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

time: 153 ms (started: 2021-08-04 18:08:56 +08:00)


最后，我们使用初始化器将预训练的嵌入加载到一个层中。 不断嵌入 为了在训练过程中不破坏预训练的表示，我们通过 trainable=False 冻结层

In [28]:
embedding_layer = layers.Embedding(max_tokens,
                                   embedding_dim,
                                   embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                   trainable=False,
                                   mask_zero=True,
                                  )

time: 10.9 ms (started: 2021-08-04 18:08:59 +08:00)


在 GloVe 嵌入之上训练一个简单的双向 LSTM 我们现在准备训练一个新模型——与我们之前的模型相同，但利用 100 维预训练的 GloVe 嵌入而不是 128 维学习嵌入。

> 清单 11.22 使用预训练嵌入层的模型

In [29]:
inputs = keras.Input(shape=(None,), dtype="int64")

embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                             save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                34048     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 92s 122ms/step - loss: 0.5806 - accuracy: 0.6931 - val_l

time: 13min 7s (started: 2021-08-04 18:09:09 +08:00)


In [30]:
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 43s 48ms/step - loss: 0.2877 - accuracy: 0.8777
Test acc: 0.878
time: 49.3 s (started: 2021-08-04 18:22:16 +08:00)


你会发现在这个特定的任务中，预训练的嵌入并不是很有帮助，因为数据集包含足够的样本，可以从头开始学习一个足够专业的嵌入空间。 但是，当您使用较小的数据集时，利用预训练嵌入可能会非常有用。

## Transformer结构

从 2017 年开始，一种新的模型架构开始在大多数自然语言处理任务中超越循环神经网络：Transformer。

在 Vaswani 等人的开创性论文“Attention is all you need”中介绍了 Transformer。 论文的要点就在标题中：事实证明，一种称为“神经注意力”的简单机制可用于构建不具有任何循环层或卷积层的强大序列模型。

这一发现在自然语言处理领域掀起了一场革命——甚至更远。 神经注意力已经迅速成为深度学习中最具影响力的思想之一。 在本节中，您将深入了解它的工作原理以及为什么它被证明对序列数据如此有效。 然后，您将利用自注意力创建一个 Transformer 编码器，这是 Transformer 架构的基本组件之一，并将其应用于 IMDB 电影评论分类任务。

### 理解自注意力

在阅读本书时，您可能会略读某些部分并专心阅读其他部分，这取决于您的目标或兴趣是什么。 如果你的模型也这样做呢？ 这是一个简单而强大的想法：并非模型看到的所有输入信息对手头的任务都同等重要，因此模型应该“多关注”某些特征，而“少关注”其他特征。

这听起来很熟悉吗？ 您已经在本书中两次遇到过类似的概念：
* convnets 中的最大池化查看空间区域中的特征池，并仅选择一个特征来保留。 这是一种“全有或全无”的注意力形式：保留最重要的特征并丢弃其余特征。
* TF-IDF 归一化根据不同令牌可能携带的信息量为令牌分配重要性分数。 重要的令牌得到提升，而不相关的令牌淡出。 这是一种持续的关注形式。

你可以想象出许多不同形式的注意力，但它们都是从计算一组特征的重要性得分开始的，对更相关的特征得分较高，对不太相关的特征得分较低。 这些分数应该如何计算，以及你应该用它们做什么，会因方法而异。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4vjhrw2wj30v60u0tb6.jpg)

至关重要的是，这种注意力机制不仅仅可以用于突出或擦除某些特征。 它可用于使功能具有上下文感知能力。 您刚刚了解了词嵌入——捕捉不同词之间语义关系“形状”的向量空间。 在嵌入空间中，单个单词具有固定位置——与空间中其他单词的一组固定关系。 但这并不是语言的运作方式：一个词的含义通常是特定于上下文的。 当您标记日期时，您不是在谈论与约会时相同的“日期”，也不是您在市场上购买的那种日期。 当你说“我很快就会见到你”时，“看到”这个词的含义与“我会看到这个项目结束”或“我明白你的意思”中的“看到”有着微妙的不同 . 当然，“他”、“它”、“在”等代词的含义完全是特定于句子的，甚至可以在一个句子中多次变化。

显然，智能嵌入空间将为一个词提供不同的向量表示，具体取决于它周围的其他词。 这就是 self-attention 的用武之地。 self-attention 的目的是通过使用序列中相关标记的表示来调整标记的表示。 这会产生上下文感知标记表示。 考虑一个例句：“火车准时离开了车站。” 现在，考虑句子中的一个词：“站”。 我们在谈论什么样的车站？ 会不会是电台？ 也许是国际空间站？ 让我们通过 self-attention 在算法上计算出来（见图 11.6）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4vm1uq6yj31b20tw438.jpg)

第 1 步：计算“station”的向量与句子中所有其他单词之间的相关性分数。这些是我们的“注意力分数”。我们将简单地使用两个词向量之间的点积来衡量它们之间的关系强度。这是一个计算效率非常高的距离函数，早在 Transformers 之前，它就已经是将两个词嵌入相互关联的标准方法。在实践中，这些分数也会经过一个缩放函数和一个 softmax，但现在，这只是一个实现细节。

第 2 步：计算句子中所有词向量的总和，由我们的相关性得分加权。与“station”密切相关的词对总和的贡献更大（包括“station”一词
本身），而无关的词几乎没有任何贡献。由此产生的向量是我们对“站”的新表示：一种结合了周围环境的表示。特别是，它包含了“火车”向量的一部分，说明它实际上是一个“火车站”。

你会对句子中的每个词重复这个过程，产生一个新的编码句子的向量序列。让我们在类似 NumPy 的伪代码中看到它：

In [33]:
def self_attention(input_sequence):
    
    output = np.zeros(shape=input_sequence.shape)
    
#     迭代输入序列中的每个标记
    for i, pivot_vector in enumerate(input_sequence): 
        scores = np.zeros(shape=(len(input_sequence),))
        
        for j, vector in enumerate(input_sequence):
#             计算令牌和其他所有令牌之间的点积（注意力分数）
            scores[j] = np.dot(pivot_vector, vector.T) 
    
#         按归一化因子缩放并应用 softmax
        scores /= np.sqrt(input_sequence.shape[1]) 
        scores = softmax(scores)
        
        
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
#             取注意力分数加权的所有标记的总和
            new_pivot_representation += vector * scores[j] 
    
#         那个总和就是我们的输出
        output[i] = new_pivot_representation 
    return output

time: 11.6 ms (started: 2021-08-04 18:27:14 +08:00)


当然，在实践中您会使用矢量化实现。 Keras 有一个内置层来处理它：MultiHeadAttention 层。 以下是您将如何使用它：

In [ ]:
num_heads = 4
embed_dim = 256

mha_layer = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

outputs = mha_layer(inputs, inputs, inputs)

读到这里，你可能想知道：
为什么将输入传递给层三次？ 这似乎是多余的。 这些“多头”指的是什么？ 这听起来很吓人——如果你剪掉它们，它们还会长出来吗？

这两个问题都有简单的答案。 让我们来看看。

**GENERALIZED SELF-ATTENTION：查询键值模型**

到目前为止，我们只考虑了一个输入序列。 然而，Transformer 架构最初是为机器翻译而开发的，您必须在其中处理两个输入序列：您当前正在翻译的源序列（例如“今天天气如何？”），以及您正在转换的目标序列 它到（例如“¿Qué tiempo hace hoy？”）。 Transformer 是一种序列到序列模型：它旨在将一个序列转换为另一个序列。 您将在本章后面深入了解序列到序列模型。

现在，让我们退后一步。 我们介绍的自注意力机制执行以下操作，示意性地：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4xtr7zjhj317u07iq3c.jpg)

这意味着：“对于输入（A）中的每个标记，计算该标记与输入（B）中的每个标记的相关程度，然后使用这些分数对输入的标记总和进行加权”。至关重要的是，没有什么需要 A 、B 和 C 指代相同的输入序列。 在一般情况下，您可以使用三个不同的序列来执行此操作。 我们将它们称为“查询”、“键”和“值”。 操作变成：“对于查询中的每个元素，计算元素与每个键的相关程度，这些使用这些分数来加权值的总和”。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4xww1sbfj318o04a3yr.jpg)

该术语来自搜索引擎和推荐系统。想象一下，您正在输入一个查询以从您的收藏中检索一张照片——“海滩上的狗”。在内部，数据库中的每张图片都由一组关键字描述——“猫”、“狗”、“派对”等。我们将这些称为“键”。搜索引擎将首先将您的查询与数据库中的键进行比较。 “Dog” 产生 1 的匹配，“cat”产生 0 的匹配。然后它会根据匹配强度（相关性）对这些键进行排名，并按照相关性的顺序返回与前 N 个匹配相关联的图片。

从概念上讲，这就是 Transformer 风格的注意力正在做的事情。你有一个描述你正在寻找的东西的参考序列：查询。你有一个你试图从中提取信息的知识体系：价值观。每个值都分配了一个键，该键以易于与查询进行比较的格式描述该值。您只需将查询与键匹配即可。然后返回值的加权总和。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4xzwrbuoj30wg0k6jsy.jpg)

实际上，键和值通常是相同的序列。 例如，在机器翻译中，查询将是目标序列，而源序列将扮演键和值的角色：对于目标的每个元素（如“tiempo”），您想回到源 （“今天的天气怎么样？”）并确定与之相关的不同位（“tiempo”和“weather”应该有很强的匹配）。 很自然地，如果你只是在做序列分类，那么查询、键和值都是一样的：你将一个序列与其自身进行比较，用整个序列的上下文来丰富每个标记。

这就解释了为什么我们需要将输入三次传递给 MultiHeadAttention 层。 但为什么要“多头”关注？

### 多头注意力

“多头注意力”是对自注意力机制的额外调整，在“注意力就是你所需要的”中介绍。 “多头”绰号是指自注意力层的输出空间被分解为一组独立的子空间，分别学习：初始查询、键和值，并通过三个独立的组发送 密集投影，产生三组调制查询、键和值，每组都通过神经网络处理
注意，三个输出连接在一起成为一个输出序列。每个这样的子空间称为“头”。

完整的图片是这样的：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4y4wrxikj317b0u0q5h.jpg)

可学习的密集投影的存在使该层能够实际学习一些东西，而不是纯粹的无状态转换，需要在它之前或之后附加层才能有用。此外，拥有独立的头有助于该层为每个令牌学习不同的特征组，其中一组内的特征彼此相关，但大部分独立于不同组中的特征。

这在原理上类似于使深度可分离卷积起作用的原理：在深度可分离卷积中，卷积的输出空间被分解为许多独立学习的子空间（每个输入通道一个）。论文“Attention is all you need”是在将特征空间分解为独立子空间的想法已被证明为计算机视觉模型提供了巨大好处的时候写的——无论是在深度可分离卷积的情况下，还是在这种情况下一种密切相关的方法，分组卷积。多头注意力只是将相同的想法应用于自注意力。

### Transformer编码器

如果添加额外的密集投影如此有用，为什么我们不也将一两个应用于注意力机制的输出？实际上，这是个好主意，让我们这样做。你知道吗，我们的模型开始做很多事情，所以我们可能想要添加残差连接，以确保我们不会在此过程中破坏任何有价值的信息——我们在第 9 章中了解到它们是对于任何足够深的架构来说都是必须的。哦，还有我们在第 9 章中学到的另一件事：归一化层应该帮助梯度更好地流动
反向传播。让我们也添加这些。

以上大致是我想象中在当时 Transformer 架构的发明者脑海中展开的思考过程。将输出分解为多个独立空间、添加残差连接、添加归一化层——所有这些都是标准架构模式，在任何复杂模型中都可以明智地加以利用。这些花里胡哨的元素共同构成了 Transformer 编码器——构成 Transformer 架构的两个关键部分之一。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4ycqkuiij30kg0qe3zs.jpg)

原始的 Transformer 架构由两部分组成：处理源序列的 Transformer 编码器和使用源序列生成翻译版本的 Transformer 解码器。 您将在一分钟内了解解码器部分。

至关重要的是，编码器部分可用于文本分类——它是一个非常通用的模块，它摄取序列并学习将其转换为更有用的表示。 让我们实现一个 Transformer 编码器，并在电影评论情感分类任务中试用它。

> 清单 11.23 作为子类层实现的 Transformer 编码器

In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim # 输入标记向量的大小
        self.dense_dim = dense_dim # 内密层尺寸
        self.num_heads = num_heads # 注意力头数
        
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, 
                                                   key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),
                                            layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        
    def call(self, inputs, mask=None): # 计算在call()中
        # 小细节：Embedding 层将生成的掩码将是 2D 的，但注意力层预计是 3D 或 4D，因此我们扩大了它的等级。
        if mask is not None:
            mask = mask[:, tf.newaxis, :] 
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self): # 实现序列化以便我们可以保存模型
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, 
                       "num_heads": self.num_heads,
                       "dense_dim": self.dense_dim,
                      })
        return config

time: 4.93 ms (started: 2021-08-05 12:43:13 +08:00)


您会注意到，我们在这里使用的标准化层与您之前在图像模型中使用过的 BatchNormalization 层不同。 这是因为 BatchNormalization 不适用于序列数据。 相反，我们使用 LayerNormalization 层，它独立于批次中的其他序列对每个序列进行标准化。 像这样，在类似 NumPy 的伪代码中：

In [35]:
def layer_normalization(batch_of_sequences): #输入形状：（batch_size、sequence_length、embedding_dim）。
    # 为了计算均值和方差，我们只在最后一个轴（轴 -1）上合并数据。
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1) 
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1) 
    
    return (batch_of_sequences - mean) / variance

time: 1.18 ms (started: 2021-08-04 19:14:54 +08:00)


与 BatchNormalization 比较（在训练期间）：

In [37]:
def batch_normalization(batch_of_images): # 输入形状：（batch_size、高度、宽度、通道）。
#     在批次轴（轴 0）上汇集数据，这会在批次中创建样本之间的相互作用。
    mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2)) 
    variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2)) 
    return (batch_of_images - mean) / variance

time: 1.42 ms (started: 2021-08-04 19:31:20 +08:00)


BatchNormalization 从多个样本中收集信息以获得特征均值和方差的准确统计数据，而 LayerNormalization 仅将每个序列内的数据单独池化，更适合序列数据。

现在我们已经实现了我们的 TransformerEncoder，我们可以使用它来组装一个类似于您之前看到的基于 GRU 的文本分类模型：

> 清单 11.24 结合了 Transformer 编码器和池化层的文本分类模型

In [38]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 由于 TransformerEncoder 返回完整序列，我们需要通过全局池化层将每个序列减少为单个向量以进行分类。
x = layers.GlobalMaxPooling1D()(x) 
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
transformer_encoder (Transfo (None, None, 256)         543776    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 5,664,033
Trainable params: 5,664,033
Non-trainable params: 0
_________________________________________________

让我们训练它。 它达到了 87.5% 的测试准确率——比 GRU 模型略差。

清单 11.25 训练和评估基于 Transformer 编码器的模型

In [39]:
callbacks = [keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                             save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 77s 117ms/step - loss: 0.5015 - accuracy: 0.7721 - val_loss: 0.3333 - val_accuracy: 0.8546
Epoch 2/20
625/625 [==============================] - 74s 118ms/step - loss: 0.3277 - accuracy: 0.8617 - val_loss: 0.2899 - val_accuracy: 0.8800
Epoch 3/20
625/625 [==============================] - 76s 121ms/step - loss: 0.2651 - accuracy: 0.8909 - val_loss: 0.2742 - val_accuracy: 0.8928
Epoch 4/20
625/625 [==============================] - 77s 123ms/step - loss: 0.2048 - accuracy: 0.9176 - val_loss: 0.3323 - val_accuracy: 0.8748
Epoch 5/20
625/625 [==============================] - 76s 121ms/step - loss: 0.1638 - accuracy: 0.9373 - val_loss: 0.3529 - val_accuracy: 0.8766
Epoch 6/20
625/625 [==============================] - 72s 115ms/step - loss: 0.1378 - accuracy: 0.9488 - val_loss: 0.4422 - val_accuracy: 0.8686
Epoch 7/20
625/625 [==============================] - 75s 120ms/step - loss: 0.1193 - accuracy: 0.9556 - val_loss: 0.4410 - val_ac


KeyboardInterrupt



In [ ]:
model = keras.models.load_model("transformer_encoder.keras",
                                custom_objects={"TransformerEncoder": TransformerEncoder}) 
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

此时，您应该开始感到有些不安。这里有些不对劲。你能说出它是什么吗？

本节表面上是关于“序列模型”的。我首先强调了词序的重要性。我说过 Transformer 是一种序列处理架构，最初是为机器翻译而开发的。然而……您刚刚看到的 Transformer 编码器根本不是序列模型。你注意到了吗？它由彼此独立处理序列标记的密集层和将标记视为一个集合的注意力层组成。你可以改变一个序列中标记的顺序，你会得到完全相同的成对注意力分数和完全相同的上下文感知表示。如果你在每篇电影评论中完全打乱单词，模型不会注意到，你仍然会得到完全相同的准确度。 Self-attention 是一种集合处理机制，专注于序列元素对之间的关​​系（见图 11.10）——它不知道这些元素是出现在序列的开头、结尾还是中间。等等，那为什么我们说Transformer是一个序列模型呢？如果它不看词序，它怎么可能对机器翻译有好处？

我在本章前面暗示了解决方案：我顺便提到 Transformer 是一种混合方法，它在技术上与顺序无关，但它会在它处理的表示中手动注入顺序信息。 这是缺少的成分！ 它被称为“位置编码”。 让我们来看看。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt4zzp1oubj316y0to41c.jpg)

**使用位置编码重新输入订单信息**

位置编码背后的想法非常简单：为了让模型访问词序信息，我们将添加到每个词嵌入单词在句子中的位置。我们的输入词嵌入将有两个组成部分：通常的词向量，它表示独立于任何特定上下文的单词，以及一个位置向量，表示单词在当前句子中的位置。希望该模型随后会弄清楚如何最好地利用这些附加信息。

你能想到的最简单的方案是将单词的位置连接到它的嵌入向量。您将向向量添加一个“位置”轴，并为序列中的第一个单词填充 0，为第二个单词填充 1，依此类推。

然而，这可能并不理想，因为您的位置可能是非常大的整数，这会破坏嵌入向量中的值范围。如您所知，神经网络不喜欢非常大的输入值或离散输入分布。

最初的“Attention is all you need paper”使用了一个有趣的技巧来编码词位置：它向词嵌入添加了一个向量，该向量包含范围内的值，该范围内的值根据位置循环变化 [-1, 1]（它使用余弦函数 为了达成这个）。 这个技巧提供了一种通过小值向量在大范围内唯一表征任何整数的方法。 这很聪明，但它不是我们将在我们的案例中使用的。 我们将做一些更简单、更有效的事情：我们将学习位置嵌入向量，就像我们学习嵌入单词索引一样。 然后我们将继续将我们的位置嵌入添加到相应的词嵌入中，以获得位置感知词嵌入。 这种技术称为“位置嵌入”。 让我们实现它：

> 清单 11.26 将位置嵌入实现为子类层

In [25]:
class PositionalEmbedding(layers.Layer):
#     位置嵌入的一个缺点是需要提前知道序列长度。
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs): 
        super().__init__(**kwargs)
#         为令牌索引准备嵌入层
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
#         另一个嵌入层用于令牌位置
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, 
                                                    output_dim=output_dim) 
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
#         将两个嵌入向量相加
        return embedded_tokens + embedded_positions
    
    
#     与嵌入层一样，该层应该能够生成掩码，以便我们可以忽略输入中的填充 0。 
#     框架将自动调用计算掩码方法，掩码将传播到下一层。
    def compute_mask(self, inputs, mask=None): 
        return tf.math.not_equal(inputs, 0)
    
#     实现序列化以便我们可以保存模型
    def get_config(self):
        config = super().get_config()
        config.update({"output_dim": self.output_dim,
                       "sequence_length": self.sequence_length,
                       "input_dim": self.input_dim,
                      })
        return config

time: 1.49 ms (started: 2021-08-05 12:42:49 +08:00)


您可以像使用常规嵌入层一样使用此 PositionEmbedding 层。 让我们看看它的实际效果！

**保存自定义图层的注意事项** 

当您编写自定义层时，请确保实现 get_config 方法：这使层能够从其 config dict 重新实例化，这在模型保存和加载期间非常有用。 该方法应返回一个 Python 字典，其中包含用于创建层的构造函数参数的值。

所有 Keras 层都可以通过以下方式序列化和反序列化：

In [ ]:
config = layer.get_config()
# 请注意，配置不包含权重值，因此层中的所有权重都从头开始初始化。
new_layer = layer.__class__.from_config(config)

例如：

In [ ]:
layer = PositionalEmbedding(sequence_length, input_dim, output_dim)
config = layer.get_config()
new_layer = PositionalEmbedding.from_config(config)

保存包含自定义层的模型时，保存文件将包含这些配置字典。 从文件加载模型时，您应该为加载过程提供自定义层类，以便它可以理解配置对象：

In [ ]:
model = keras.models.load_model(filename, custom_objects={"PositionalEmbedding": PositionalEmbedding})

**综合起来：文本分类转换器**

要开始考虑词序，您所要做的就是将旧层与嵌入我们的位置感知版本交换：

> 清单 11.27 结合了位置嵌入、Transformer 编码器和池化层的文本分类模型

In [44]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

# 看这里！
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs) 
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                             save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
positional_embedding (Positi (None, None, 256)         5273600   
_________________________________________________________________
transformer_encoder_1 (Trans (None, None, 256)         543776    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 5,817,633
Trainable params: 5,817,633
Non-trainable params: 0
_________________________________________________

KeyboardInterrupt: 

time: 19min 3s (started: 2021-08-04 20:15:12 +08:00)


In [ ]:
model = keras.models.load_model("full_transformer_encoder.keras",
                                custom_objects={"TransformerEncoder": TransformerEncoder,
                                                "PositionalEmbedding": PositionalEmbedding})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

我们达到了 88.3% 的测试准确率，这是一个坚实的改进，清楚地证明了词序信息对于文本分类的价值。 这是迄今为止我们最好的序列模型！ 然而，它仍然比词袋方法低一个档次。

### 何时在词袋模型上使用序列模型？

您有时可能会听到词袋方法已经过时，并且无论您正在查看什么任务或数据集，基于 Transformer 的序列模型都是可行的方法。 这绝对不是这种情况：在许多情况下，在二元组袋顶部的一小堆层仍然是一种完全有效且相关的 Dense 方法。 事实上，在本章中我们在 IMDB 数据集上尝试的各种技术中，迄今为止表现最好的是 bag-of-bigrams！

那么，什么时候您应该更喜欢一种方法而不是另一种方法？

2017 年，我和我的团队对各种文本分类技术在许多不同类型的文本数据集上的性能进行了系统分析，我们发现了一个非凡且令人惊讶的经验法则来决定是否使用词袋模型 或序列模型。 各种黄金常数。

事实证明，在处理新的文本分类任务时，您应该密切注意训练数据中的样本数与每个样本的平均单词数之间的比率。 如果该比率很小（小于 1,500），那么bag-of-bigrams 模型的性能会更好（而且作为奖励，它的训练和迭代速度也会快得多）。 如果该比率高于 1,500，那么您应该使用序列模型。 换句话说，当有大量训练数据可用且每个样本相对较短时，序列模型效果最好。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt50v7pcsoj31as0iodi0.jpg)

因此，如果您要对 1,000 字长的文档进行分类，并且其中有 100,000 个，那么您应该使用二元模型（比率：100）。 如果您对平均 40 字长的推文进行分类，并且您有 50,000 条推文，那么您还应该使用二元模型（比率：1,250）。 但是，如果您将数据集大小增加到 500,000 条推文，则使用 Transformer 编码器（比率：12,500）。 IMDB 影评分类任务怎么样？ 我们有 20,000 个训练样本，平均字数为 233，因此我们的经验法则指向二元模型——这证实了我们在实践中的发现。

因此，如果您要对 1,000 字长的文档进行分类，并且其中有 100,000 个，那么您应该使用二元模型（比率：100）。 如果您对平均 40 字长的推文进行分类，并且您有 50,000 条推文，那么您还应该使用二元模型（比率：1,250）。 但是，如果您将数据集大小增加到 500,000 条推文，则使用 Transformer 编码器（比率：12,500）。 IMDB 影评分类任务怎么样？ 我们有 20,000 个训练样本，平均字数为 233，因此我们的经验法则指向二元模型——这证实了我们在实践中的发现。

这在直觉上是有道理的：序列模型的输入代表一个更丰富、更复杂的空间，因此需要更多的数据来绘制该空间——同时，一组简单的术语是一个非常简单的空间，您可以训练逻辑回归 最重要的是仅使用数百或数千个样本。 此外，样本越短，模型丢弃它包含的任何信息的能力就越少——尤其是词序变得更加重要，丢弃它会产生歧义。 句子“这部电影是炸弹”和“这部电影是炸弹”具有非常接近的一元表示，这可能会混淆词袋模型，但序列模型可以分辨哪个是负面的，哪个是正面的。 使用更长的样本，单词统计将变得更加可靠，并且仅从单词直方图中可以更明显地看出主题或情绪。

现在，请记住，这个启发式规则是专门为文本分类开发的，它可能不一定适用于其他 NLP 任务——例如，在机器翻译方面，与 RNN 相比，Transformer 特别适用于非常长的序列。 我们的启发式也只是一个经验法则，而不是科学定律，所以希望它在大多数时间都有效，但不一定每次都有效。

## 超越文本分类：序列到序列学习

到目前为止，您已经拥有处理大多数自然语言处理任务所需的所有工具。 但是，到目前为止，您只有这些工具可以解决一个问题：文本分类。 这是一个非常流行的用例，但 NLP 的分类还有很多。 在本节中，您将通过学习序列到序列模型来加深您的专业知识。

序列到序列模型将序列作为输入（通常是句子或段落）并将其翻译成不同的序列。 这是许多 NLP 最成功应用的核心任务，例如（见图 11.12）：
* 机器翻译：将源语言中的段落转换为目标语言中的对应段落。
* 文本摘要：将长文档转换为保留最重要信息的较短版本。
* 问答：将输入的问题转换为其答案。 聊天机器人：将对话提示转换为对此提示的回复，或将对话的历史记录转换为对话中的下一个回复。
* 文本生成：将文本提示转换为完成提示的段落。
* 等等。

图 11.12 描述了序列到序列模型背后的通用模板。 培训期间：
* 编码器模型将源序列转换为中间表示。
* 通过查看 i 个先前的标记（0 到 i - 1）和编码的源序列，解码器被训练来预测目标序列中的下一个标记。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt515nklwsj317j0u0n14.jpg)

在推理过程中，我们无法访问目标序列——我们试图从头开始预测它。 我们必须一次生成一个令牌：
* 首先，我们从编码器获得编码的源序列。
* 解码器首先查看编码的源序列以及初始“种子”标记（例如 string ），并使用它们来预测“[start]”序列中的第一个真实标记。
* 到目前为止的预测序列被反馈到解码器，解码器生成下一个标记，依此类推，直到生成停止标记（例如字符串“[end]”）。

到目前为止，您学到的所有知识都可以重新用于构建这种新型模型。 让我们深入了解。

### 机器翻译示例

我们将在机器翻译任务上演示序列到序列建模。 机器翻译正是 Transformer 的开发目的！ 我们将从循环序列模型开始，然后跟进完整的 Transformer 架构。

我们将使用 Anki (www.manythings.org/anki/) 提供的英语到西班牙语翻译数据集。 让我们下载它：

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

该文本文件每行包含一个示例：一个英语句子，后跟一个制表符，后跟相应的西班牙语句子。 让我们解析这个文件。

In [5]:
text_file = "data/spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
    
text_pairs = []

# 迭代文件中的行。
for line in lines:
#     每行包含一个英语短语及其西班牙语翻译，以制表符分隔。
    english, spanish = line.split("\t")
#     我们在西班牙语句子前面加上“[start]”和“[end]”，以匹配图 11.12 中的模板。
    spanish = "[start] " + spanish + " [end]" 
    text_pairs.append((english, spanish))

time: 525 ms (started: 2021-08-05 11:15:31 +08:00)


我们的 text_pairs 看起来像这样：

In [6]:
import random
print(random.choice(text_pairs))

('You might want to have someone look into that matter.', '[start] Puede que quieras que alguien investigue ese asunto. [end]')
time: 1.94 ms (started: 2021-08-05 11:15:33 +08:00)


让我们将它们打乱，然后分成通常的训练、验证和测试集：

In [7]:
import random

random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

time: 195 ms (started: 2021-08-05 11:15:35 +08:00)


接下来，让我们准备两个单独的 TextVectorization 层，一个用于英语，一个用于西班牙语。 我们将需要自定义字符串的预处理方式：
* 我们需要保留我们插入的“[start]”和“[end]”标记。 默认情况下，字符 [and] 将被删除，但我们希望保留它们以便我们可以区分单词“start”和开始标记“[start]”。
* 标点符号因语言而异！ 在西班牙语 TextVectorization 层中，如果我们要去除标点字符，我们还需要去除字符。

请注意，对于非玩具翻译模型，我们会将标点字符视为单独的标记而不是剥离它们，因为我们希望能够生成标点正确的句子。 在我们的例子中，为简单起见，我们将去掉所有标点符号。

> 清单 11.28 向量化英语和西班牙语文本对

In [8]:
from keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf
import string
import re

# 为西班牙语 TextVectorization 层准备一个自定义字符串标准化函数：
# 它保留并删除（以及来自 strings.punctuation 的所有其他 [ ] ¿ 字符）。
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# 为简单起见，我们将只查看每种语言的前 15,000 个单词，并将句子限制为 20 个单词。
vocab_size = 15000
sequence_length = 20

# 英语层
source_vectorization = TextVectorization( 
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    )

# 西班牙语层
target_vectorization = TextVectorization( 
    max_tokens=vocab_size,
    output_mode="int",
#     生成具有一个额外标记的西班牙语句子，因为我们需要在训练期间将句子偏移一级。
    output_sequence_length=sequence_length + 1, 
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

# 学习每种语言的词汇。
source_vectorization.adapt(train_english_texts) 
target_vectorization.adapt(train_spanish_texts)

time: 1min 22s (started: 2021-08-05 11:15:43 +08:00)


最后，我们可以将我们的数据变成一个管道。 我们希望它返回一个元组 tf.data (inputs, target) input 其中是一个带有两个键的字典，“encoder_inputs”（英语句子）和“decoder_inputs”（西班牙语句子），并且是偏移一的西班牙语句子 前进目标。

> 清单 11.29 为翻译任务准备训练和验证数据集

In [9]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({"english": eng,
             "spanish": spa[:, :-1], #输入的西班牙语句子不包括最后一个标记，以保持输入和目标的长度相同。
            }, spa[:, 1:]) # 目标西班牙语句子领先一步。 两者的长度仍然相同（20 个字）。

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache() #使用内存缓存来加速预处理。

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

time: 2.91 s (started: 2021-08-05 11:19:37 +08:00)


这是我们的数据集输出的样子：

In [10]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)
time: 1.64 s (started: 2021-08-05 11:20:39 +08:00)


数据现已准备就绪，是时候构建一些模型了。 在转到 Transformer 之前，我们将从循环序列到序列模型开始。

### 使用 RNN 进行序列到序列学习

从 2015 年到 2017 年，循环神经网络主导了序列到序列学习，之后被 Transformer 超越。 它们是许多现实世界机器翻译系统的基础——如第 10 章所述，大约 2017 年的谷歌翻译由七个大型 LSTM 层的堆栈提供支持。 今天仍然值得学习这种方法，因为它提供了一个简单的切入点来理解序列到序列模型。

使用 RNN 将序列转换为另一个序列的最简单、天真的方法是在每个时间步保持 RNN 的输出——在 Keras 中，它看起来像这样：

In [13]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = layers.Input(shape=(sequence_length,), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)

time: 1.16 s (started: 2021-08-05 11:24:19 +08:00)


但是，这种方法有两个主要问题：
* 目标序列必须始终与源序列长度相同。 在实践中，这种情况很少发生。 从技术上讲，这并不重要，因为您始终可以填充源序列或目标序列以使其长度匹配。
* 由于 RNN 的循序渐进特性，该模型将只查看源序列中的标记 0…N，以便预测目标序列中的标记 N。 这种限制使得这种设置不适用于大多数任务，尤其是翻译。 考虑将“今天天气很好”翻译成法语——那就是“Il fait beau aujourd’hui”。 您需要能够仅从“The”预测“Il”，仅从“The weather”等预测“Il fait”，这是不可能的。

如果你是一名人工翻译，你会先阅读整个源句子，然后再开始翻译。 如果您正在处理具有截然不同的词序的语言，例如英语和日语，这一点尤其重要。 而这正是标准序列到序列模型所做的。

在正确的序列到序列设置中（参见图 11.13），您将首先使用 RNN（编码器）将整个源序列转换为单个向量（或一组向量）。 这可能是 RNN 的最后一个输出，或者是其最终的内部状态向量。 然后你可以使用这个向量（或多个向量）作为另一个 RNN（解码器）的 ，它会查看目标序列中的初始状态元素 0… N，并尝试预测目标序列中的第 N+1 步。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt5r4g63ksj31c20m6tbq.jpg)

让我们使用基于 GRU 的编码器和解码器在 Keras 中实现这一点。 选择 GRU 而不是 LSTM 让事情变得更简单，因为 GRU 只有一个状态向量，而 LSTM 有多个。 让我们从编码器开始：

> 清单 11.30 基于 GRU 的编码器

In [16]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

# 英文源句放在这里。 指定输入的名称使我们能够使用具有输入字典的 fit() 模型。
source = keras.Input(shape=(None,), dtype="int64", name="english") 
# 不要忘记屏蔽：这在此设置中至关重要。
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source) 
# 我们编码的源语句是双向 GRU 的最后一个输出。
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

time: 6.33 s (started: 2021-08-05 11:31:45 +08:00)


接下来，让我们添加解码器——一个简单的 GRU 层，它将编码的源语句作为初始状态。 最重要的是，我们添加了一个 Dense 层，为每个输出步骤生成西班牙语词汇表的概率分布。

> 清单 11.31 基于 GRU 的解码器和端到端模型

In [17]:
# 西班牙语目标句放在这里。
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish") 
# 不要忘记掩码。
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target) 
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
# 编码的源语句作为解码器 GRU 的初始状态。
x = decoder_gru(x, initial_state=encoded_source) 
x = layers.Dropout(0.5)(x)
# 预测下一个令牌。
target_next_step = layers.Dense(vocab_size, activation="softmax")(x) 
# 端到端模型：将源句和目标句映射到未来一步的目标句。
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

time: 3.43 s (started: 2021-08-05 11:37:50 +08:00)


在训练期间，解码器将整个目标序列作为输入，但由于 RNN 的逐步性质，它只查看输入中的标记 0… N 来预测输出中的标记 N（对应于下一个 序列中的标记，因为输出旨在偏移一个步骤）。 这意味着我们只使用过去的信息来预测未来——我们应该这样做，否则，我们会作弊，我们的模型在推理时将无法工作。

让我们开始训练：

> 清单 11.32 训练我们的循环序列到序列模型

In [18]:
seq2seq_rnn.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 222s 141ms/step - loss: 1.6373 - accuracy: 0.4182 - val_loss: 1.3037 - val_accuracy: 0.5109
Epoch 2/15
1302/1302 [==============================] - 176s 135ms/step - loss: 1.3139 - accuracy: 0.5294 - val_loss: 1.1425 - val_accuracy: 0.5719
Epoch 3/15
1302/1302 [==============================] - 174s 133ms/step - loss: 1.1727 - accuracy: 0.5779 - val_loss: 1.0652 - val_accuracy: 0.6032
Epoch 4/15
1302/1302 [==============================] - 173s 133ms/step - loss: 1.0830 - accuracy: 0.6096 - val_loss: 1.0293 - val_accuracy: 0.6204
Epoch 5/15
1302/1302 [==============================] - 173s 133ms/step - loss: 1.0349 - accuracy: 0.6340 - val_loss: 1.0152 - val_accuracy: 0.6309
Epoch 6/15
1302/1302 [==============================] - 173s 133ms/step - loss: 1.0050 - accuracy: 0.6518 - val_loss: 1.0136 - val_accuracy: 0.6363
Epoch 7/15
1302/1302 [==============================] - 173s 133ms/step - loss: 0.9854 - accuracy: 0.6650 - val_

time: 44min 24s (started: 2021-08-05 11:39:36 +08:00)


我们选择准确性作为在训练期间监控验证集性能的粗略方法。 我们达到了 64% 的准确率：平均而言，该模型在 64% 的时间内正确预测了西班牙语句子中的下一个单词。 然而，在实践中，下一个标记的准确性并不是机器翻译模型的重要指标，特别是因为它假设在预测标记 N+1 时已经知道从 0 到 N 的正确目标标记——实际上，在 推断，您是从头开始生成目标句子，因此您不能依赖先前生成的标记是 100% 正确的。 如果您继续在现实世界的机器翻译系统上工作，您可能会使用 BLEU 分数来评估您的模型——一种查看整个生成序列的指标，并且似乎与人类对翻译质量的感知密切相关。

最后，让我们使用我们的模型进行推理——我们将在测试集中挑选一些句子并检查我们的模型如何翻译它们。 我们从种子标记“[start]”开始，并将其与编码的英文源句子一起输入解码器模型。 我们检索下一个令牌预测。 我们将其重新注入解码器，重复，在每次迭代中采样一个新的目标标记，直到我们到达“[end]”或达到最大句子长度。

> 清单 11.33 使用我们的 RNN 编码器和解码器翻译新句子

In [22]:
import numpy as np

# 准备一个 dict 将令牌索引预测转换为字符串令牌
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab)) 

max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]" #种子令牌
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
    #     采样下一个令牌
        next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, 
                                                      tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :]) 
        sampled_token = spa_index_lookup[sampled_token_index]
    #     将下一个标记预测转换为字符串并将其附加到生成的句子中
        decoded_sentence += " " + sampled_token
    #     退出条件：达到最大长度或采样一个停止字符。
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
He is still in bed.
[start] Él está en cama [end]
-
The hotel has a pleasant atmosphere.
[start] el hotel tiene una [UNK] [end]
-
I don't believe such things exist.
[start] no creo que las cosas de cosas [end]
-
What time should I go to the airport?
[start] a qué hora debería ir al aeropuerto [end]
-
His new book will appear next month.
[start] su nuevo libro el día que viene [end]
-
We seem to be trapped.
[start] parece estar en lo que me [UNK] [end]
-
I can do it in half the time.
[start] puedo hacerlo en la hora [end]
-
The boss is an open person.
[start] el hombre es una persona muy difícil [end]
-
Send him in.
[start] [UNK] en él [end]
-
I don't speak Swedish.
[start] no hablo nada [end]
-
Can he speak Japanese?
[start] Él sabe hablar japonés [end]
-
Do you think I should go by myself?
[start] crees que debería ir solo [end]
-
I still love this bicycle.
[start] todavía me encanta esta bicicleta [end]
-
Picasso painted this picture in 1950.
[start] [UNK] este [UNK] de la en la ma

请注意，这种推理设置虽然非常简单，但效率相当低，因为我们每次对新单词进行采样时都会重新处理整个源句子和整个生成的目标句子。 在实际应用中，您将编码器和解码器分解为两个独立的模型，并且您的解码器在每次令牌采样迭代时只运行一个步骤，重用其先前的内部状态。

这是我们的翻译结果。 对于玩具模型，它工作得很好，尽管它仍然会犯许多基本错误，如清单 11.34 所示。

> 清单 11.34 循环翻译模型的一些示例结果

In [ ]:
Who is in this room?
[start] quién está en esta habitación [end]
-
That doesn't sound too dangerous.
[start] eso no es muy difícil [end]
-
No one will stop me.
[start] nadie me va a hacer [end]
-
Tom is friendly.
[start] tom es un buen [UNK] [end]

有很多方法可以改进这个玩具模型：我们可以为编码器和解码器使用一个深层的循环层（请注意，对于解码器，这会使状态管理更加复杂）。我们可以使用 LSTM 代替 GRU。等等。然而，除了这些调整之外，用于序列到序列学习的 RNN 方法还有一些基本的局限性：
* 源序列表示必须完全保存在编码器状态向量中，这对您可以翻译的句子的大小和复杂性产生了重大限制。这有点像人类完全从记忆中翻译一个句子，在生成翻译时没有看两次源句子。
* RNN 无法处理很长的序列，因为它们往往会逐渐忘记过去——当你到达任一序列中的第 100 个标记时，关于序列开始的信息就很少了。这意味着基于 RNN 的模型无法保持长期上下文，这对于翻译长文档至关重要。

这些限制导致机器学习社区采用 Transformer 架构来解决序列到序列问题。 让我们来看看。

### 使用 Transformer 进行序列到序列学习

序列到序列学习是 Transformer 真正闪耀的任务。神经注意力使 Transformer 模型能够成功处理比 RNN 可以处理的更长、更复杂的序列。

作为一名将英语翻译成西班牙语的人，你不会一次一个单词读一个英语句子，记住它的意思，然后一次一个单词生成西班牙语句子。这可能适用于五个字的句子，但不太可能适用于整个段落。相反，您可能希望在源句子和正在进行的翻译之间来回切换，并在写下翻译的不同部分时“注意”源中的不同单词。

这正是您可以通过神经注意力和 Transformers 实现的目标。您已经熟悉 Transformer 编码器，它使用自注意力来生成输入序列中每个标记的上下文感知表示。在序列到序列的 Transformer 中，Transformer 编码器自然会扮演编码器的角色，它读取源序列并生成它的编码表示。然而，与我们之前的 RNN 编码器不同，Transformer 编码器将编码表示保持在序列格式中：它是一个上下文感知嵌入向量的序列。

模型的后半部分是 Transformer 解码器。 就像 RNN 解码器一样，它读取目标序列中的标记 0… N 并尝试预测标记 N+1。 至关重要的是，在执行此操作时，它使用神经注意力来识别编码源语句中的哪些标记与它当前试图预测的目标标记最密切相关——这可能与人工翻译会做的没什么不同。 回想一下查询-键-值模型：在 Transformer 解码器中，目标序列作为一个注意力“查询”，用于更密切地关注源序列的不同部分（源序列同时扮演着键和 值）。

**变压器解码器**

图 11.14 显示了完整的序列到序列转换器。 查看解码器内部结构：您会发现它看起来与 Transformer 编码器非常相似，只是在应用于目标序列的自注意块和出口块的密集层之间插入了一个额外的注意块。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt5siz8oh3j30vi0t8din.jpg)

让我们实施它。 与 TransformerEncoder 一样，我们将使用 Layer 子类。 在我们关注 call() 方法（动作发生的地方）之前，让我们首先定义类构造函数，其中包含我们将需要的层。

> 清单 11.35 TransformerDecoder

In [21]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),
                                            layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        
#         此属性确保该层将其输入掩码传播到其输出：Keras 中的掩码是明确选择的。 
#         如果您将掩码传递给未实现 compute_mask() 且未公开此 supports_masking 属性的图层，则会出现错误。
        self.supports_masking = True 
    
    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim,
                       "num_heads": self.num_heads,
                       "dense_dim": self.dense_dim,})
        return config

time: 1.65 ms (started: 2021-08-05 12:26:55 +08:00)


call() 方法几乎是图 11.14 中连接图的直接呈现。 但是还有一个额外的细节我们需要考虑：因果填充。 因果填充对于成功训练序列到序列转换器绝对至关重要。 与 RNN 不同，它一次查看其输入一个步骤，因此只能访问步骤 0..N 以生成输出步骤 N（即目标序列中的标记 N+1），TransformerDecoder 是 order- 不可知：它一次查看整个目标序列。 如果允许它使用其整个输入，它会简单地学习将输入步骤 N+1 复制到输出中的位置 N。 因此，该模型将达到完美的训练精度，但当然，在运行推理时，它完全没有用，因为超过 N 的输入步骤不可用。

解决方法很简单：我们将屏蔽成对注意力矩阵的上半部分，以防止模型关注来自未来的信息——生成目标时只应使用来自目标序列中标记 0..N 的信息 令牌 N+1（参见图 TODO）。 为此，我们将向 TransformerEncoder 添加一个 get_causal_attention_mask(self,inputs) 方法以检索我们可以传递给 MultiHeadAttention 层的注意力掩码。

> 清单 11.36 生成“因果掩码”的 TransformerDecoder 方法

In [20]:
def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    
#     生成形状矩阵（sequence_length，sequence_length），一半为 1，另一半为 0
    mask = tf.cast(i >= j, dtype="int32")
    
#     沿批处理轴复制它以获得形状矩阵 (batch_size,序列长度，序列长度）
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) 
    mult = tf.concat([tf.expand_dims(batch_size, -1),
                      tf.constant([1, 1], dtype=tf.int32)], axis=0)
    return tf.tile(mask, mult)

time: 1.4 ms (started: 2021-08-05 12:26:50 +08:00)


现在，我们可以写下实现解码器前向传递的完整方法：

> 清单 11.37 Transformer Decoder 的前向传递

In [23]:
def call(self, inputs, encoder_outputs, mask=None):
#     检索因果掩码
    causal_mask = self.get_causal_attention_mask(inputs) 
    # 准备输入掩码（描述目标序列中的填充位置）
    if mask is not None:
        padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        # 将两个蒙版合并在一起
        padding_mask = tf.minimum(padding_mask, causal_mask) 
    attention_output_1 = self.attention_1(query=inputs, 
                                          value=inputs,
                                          key=inputs,
                                          # 将因果掩码传递给第一个注意力层，它对目标序列执行自注意力
                                          attention_mask=causal_mask)
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(query=attention_output_1,
                                          value=encoder_outputs,
                                          key=encoder_outputs,
                                          # 将组合掩码传递给第二个注意力层，它将源序列与目标序列相关联
                                          attention_mask=padding_mask,)
    attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    
    return self.layernorm_3(attention_output_2 + proj_output)

time: 1.48 ms (started: 2021-08-05 12:37:11 +08:00)


In [33]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),
                                            layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        
#         此属性确保该层将其输入掩码传播到其输出：Keras 中的掩码是明确选择的。 
#         如果您将掩码传递给未实现 compute_mask() 且未公开此 supports_masking 属性的图层，则会出现错误。
        self.supports_masking = True 
    
    def call(self, inputs, encoder_outputs, mask=None):
    #     检索因果掩码
        causal_mask = self.get_causal_attention_mask(inputs) 
        # 准备输入掩码（描述目标序列中的填充位置）
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            # 将两个蒙版合并在一起
            padding_mask = tf.minimum(padding_mask, causal_mask) 
        attention_output_1 = self.attention_1(query=inputs, 
                                              value=inputs,
                                              key=inputs,
                                              # 将因果掩码传递给第一个注意力层，它对目标序列执行自注意力
                                              attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1,
                                              value=encoder_outputs,
                                              key=encoder_outputs,
                                              # 将组合掩码传递给第二个注意力层，它将源序列与目标序列相关联
                                              attention_mask=padding_mask,)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)

        return self.layernorm_3(attention_output_2 + proj_output)    

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)

    #     生成形状矩阵（sequence_length，sequence_length），一半为 1，另一半为 0
        mask = tf.cast(i >= j, dtype="int32")

    #     沿批处理轴复制它以获得形状矩阵 (batch_size,序列长度，序列长度）
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) 
        mult = tf.concat([tf.expand_dims(batch_size, -1),
                          tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)    
    
    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim,
                       "num_heads": self.num_heads,
                       "dense_dim": self.dense_dim,})
        return config

time: 4.13 ms (started: 2021-08-05 12:51:35 +08:00)


**综合：机器翻译的变形金刚**

端到端 Transformer 是我们将要训练的模型，它将到目前为止的源序列和目标序列映射到未来一步的目标序列。 它直接将我们迄今为止构建的部分组合在一起：PositionalEmbedding 层、TransformerEncoder 和 TransformerDecoder。 请注意，这两个
TransformerEncoder TransformerDecoder 和它们是形状不变的，因此您可以将它们中的许多堆叠起来以创建更强大的编码器或解码器。 在我们的示例中，我们将坚持每个实例。

> 清单 11.38 端到端变压器

In [34]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# 编码源语句
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) 

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# 对目标句进行编码，并与编码后的源句组合
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs) 
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x) 

# 为每个输出位置预测一个单词
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

time: 2.28 s (started: 2021-08-05 12:51:38 +08:00)


我们现在准备训练我们的模型——我们达到了 67% 的准确率，比基于 GRU 的模型高很多。

> 清单 11.39 训练序列到序列转换器

In [ ]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 165s 120ms/step - loss: 1.0772 - accuracy: 0.6267 - val_loss: 1.0144 - val_accuracy: 0.6330
Epoch 2/30
1302/1302 [==============================] - 152s 116ms/step - loss: 1.0348 - accuracy: 0.6454 - val_loss: 0.9956 - val_accuracy: 0.6438
Epoch 3/30
1302/1302 [==============================] - 155s 119ms/step - loss: 1.0077 - accuracy: 0.6595 - val_loss: 0.9856 - val_accuracy: 0.6508
Epoch 4/30
1302/1302 [==============================] - 156s 120ms/step - loss: 0.9875 - accuracy: 0.6712 - val_loss: 0.9809 - val_accuracy: 0.6535
Epoch 5/30
1302/1302 [==============================] - 152s 117ms/step - loss: 0.9688 - accuracy: 0.6807 - val_loss: 0.9829 - val_accuracy: 0.6548
Epoch 6/30
1302/1302 [==============================] - 153s 118ms/step - loss: 0.9530 - accuracy: 0.6894 - val_loss: 0.9798 - val_accuracy: 0.6583
Epoch 7/30
1302/1302 [==============================] - 154s 118ms/step - loss: 0.9382 - accuracy: 0.6968 - val_

最后，让我们尝试使用我们的模型从测试集中翻译从未见过的英语句子。 该设置与我们用于序列到序列 RNN 模型的设置相同。

> 清单 11.40 使用我们的 Transformer 模型翻译新句子

In [ ]:
import numpy as np
    spa_vocab = target_vectorization.get_vocabulary()
    spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
    max_decoded_sentence_length = 20
    
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
        [decoded_sentence])[:, :-1]
#         采样下一个令牌
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence]) 
        sampled_token_index = np.argmax(predictions[0, i, :])
        # 将下一个标记预测转换为字符串并将其附加到生成的句子中
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # 退出条件
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

主观上，Transformer 的性能似乎明显优于基于 GRU 的翻译模型。 它仍然是一个玩具模型，但它是一个更好的玩具模型。

> 清单 11.41 Transformer 翻译模型的一些示例结果

In [ ]:
This is a song I learned when I was a kid.
# 虽然源句子没有性别化，但这个翻译假设是男性说话者。 
# 请记住，翻译模型通常会对其输入数据做出无根据的假设，从而导致算法偏差。 
# 在最坏的情况下，模型可能会产生与当前正在处理的数据无关的记忆信息的幻觉。
[start] esta es una canción que aprendí cuando era chico [end] 
-
She can play the piano.
[start] ella puede tocar piano [end]
-
I'm not who you think I am.
[start] no soy la persona que tú creo que soy [end]
-
It may have rained a little last night.
[start] puede que llueve un poco el pasado [end]

关于自然语言处理的本章到此结束——您刚刚从最基本的知识发展到了一个可以将英语翻译成西班牙语的成熟 Transformer。 教机器理解语言是您可以添加到收藏中的最新超能力。

## 章节总结

* 有两种 NLP 模型：词袋模型，处理词集或 n-gram 而不考虑它们的顺序，以及序列模型，处理词序。词袋模型由密集层组成，而序列模型可以是 RNN、1D convnet 或 Transformer。
* 在文本分类方面，训练数据中的样本数与每个样本的平均词数之间的比率可以帮助您确定应该使用词袋模型还是序列模型。
* 词嵌入是向量空间，其中词之间的语义关系被建模为表示这些词的向量之间的距离关系。
* 序列到序列学习是一个通用的、强大的学习框架，可用于解决许多 NLP 问题，包括机器翻译。序列到序列模型由处理源序列的编码器和试图通过在编码器处理的源序列的帮助下查看过去的标记来预测目标序列中未来标记的解码器组成。
* 神经注意力是一种创建上下文感知词表示的方法。它是 Transformer 架构的基础。
* Transformer 架构由一个 TransformerEncoder 和一个 TransformerDecoder 组成，在序列到序列任务上产生了出色的结果。前半部分 TransformerEncoder 也可用于文本分类或任何类型的单输入 NLP 任务。